In [1]:
import os
import pandas as pd
import numpy as np
import re
from datetime import date, datetime, timedelta
import warnings
import glob 
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Desactiva las advertencias
warnings.filterwarnings('ignore')

In [69]:
def cargar_dataframe_ambulatorio(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')

def cargar_dataframe_ieeh(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')



def filtro_dataframe_ca_tiroides_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-! asociados al cáncer de tiroides
    patron_cancer_tiroides = r'^C73'

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de tiroides
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_tiroides, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de tiroides en texto libre
    #patron_diagnostico_tiroides = re.compile(r'\bc[a-zá-ú]*[ \.]*tiroide[a-zá-ú]*\b', re.IGNORECASE)
    patron_diagnostico_tiroides = re.compile(r'\bc(?!o)[a-zá-ú]*[ \.]*tiroide(?!o)[a-zá-ú]*\b', re.IGNORECASE)

    patron_carci_in_situ = re.compile(r'carci.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno = re.compile(r'tumor.*maligno.*glandula tiroides|tumor.*maligno.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_carci_in_situ_tiroides = re.compile(r'carci.*in\s*situ.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_nodulos_tirad_4_5 = re.compile(r'n[oó]dulos.*tirad\s*[5]', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de tiroides en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_tiroides, na=False) |
        x.str.contains(patron_carci_in_situ, na=False) |
        x.str.contains(patron_tumor_maligno, na=False) |
        x.str.contains(patron_carci_in_situ_tiroides, na=False) |
        x.str.contains(patron_nodulos_tirad_4_5, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado

def filtro_dataframe_ca_colorectal_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-10 asociados al cáncer colorrectal
    patron_cancer_colorrectal = r'^C18.*|C19.*|C20.*'
    
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer colorrectal
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_colorrectal, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer colorrectal en texto libre
    patron_diagnostico_colorrectal = re.compile(r'\b(?:c[aáx[^mn]]*[ \.]*col[oa]n|\bc[aáx[^mn]]*[ \.]*rect[oa])\b', re.IGNORECASE)


    patron_adenocarcinoma_colorrectal = re.compile(r'\badenocarcinoma.*rect[a-zá-ú]*', re.IGNORECASE)
    patron_carcinoma_escamoso_colorrectal = re.compile(r'\bcarcinoma.*escamoso.*rect[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno_colon = re.compile(r'\btumor.*maligno.*colon', re.IGNORECASE)
    patron_tumor_maligno_recto = re.compile(r'\btumor.*maligno.*recto', re.IGNORECASE)
    patron_neoplasia_maligna_colon = re.compile(r'\bneoplasia.*maligna.*colon', re.IGNORECASE)
    patron_neoplasia_maligna_recto = re.compile(r'\bneoplasia.*maligna.*recto', re.IGNORECASE)
    patron_cancer_colorrectal_metastasico = re.compile(r'\bc[a-zá-ú]*[ \.]*rect[a-zá-ú]*.*metast[a-zá-ú]*', re.IGNORECASE)
    patron_cancer_colon_avanzado = re.compile(r'\bc[a-zá-ú]*[ \.]*colon.*avanzado', re.IGNORECASE)
    patron_cancer_recto_avanzado = re.compile(r'\bc[a-zá-ú]*[ \.]*rect[a-zá-ú]*.*avanzado', re.IGNORECASE)


    # Términos acompañados por otros
    patron_neoplasia_ubicaciones = re.compile(r'\bneoplasia.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_adenoca_ubicaciones = re.compile(r'\badenoca.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_gist_ubicaciones = re.compile(r'\bgist.*(?:colon\b|recto\b|rectal|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_cancer_ubicaciones = re.compile(r'\bca[aáx[^mn](?:ncer)?.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_tumor_ubicaciones = re.compile(r'\btu(?:mor)?.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer colorrectal en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_colorrectal, na=False) |
        x.str.contains(patron_adenocarcinoma_colorrectal, na=False) |
        x.str.contains(patron_carcinoma_escamoso_colorrectal, na=False) |
        x.str.contains(patron_tumor_maligno_colon, na=False) |
        x.str.contains(patron_tumor_maligno_recto, na=False) |
        x.str.contains(patron_neoplasia_maligna_colon, na=False) |
        x.str.contains(patron_neoplasia_maligna_recto, na=False) |
        x.str.contains(patron_cancer_colorrectal_metastasico, na=False) |
        x.str.contains(patron_cancer_colon_avanzado, na=False) |
        x.str.contains(patron_cancer_recto_avanzado, na=False) |
        x.str.contains(patron_neoplasia_ubicaciones, na=False) |
        x.str.contains(patron_adenoca_ubicaciones, na=False) |
        x.str.contains(patron_gist_ubicaciones, na=False) |
        x.str.contains(patron_cancer_ubicaciones, na=False) |
        x.str.contains(patron_tumor_ubicaciones, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado

def filtro_dataframe_ca_gastrico_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Patrón para identificar los códigos CIE-10 asociados al cáncer gástrico
    patron_cancer_gastrico = r'^C16.*'

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer gástrico
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_gastrico, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer gástrico en texto libre
    patron_adenocarcinoma_gastrico = re.compile(r'adenocarcinoma.*g[aá]strico', re.IGNORECASE)
    patron_carcinoma_gastrico = re.compile(r'carcinoma.*g[aá]strico', re.IGNORECASE)
    patron_linfoma_gastrico = re.compile(r'linfoma.*g[aá]strico', re.IGNORECASE)
    patron_sarcoma_gastrico = re.compile(r'sarcoma.*g[aá]strico', re.IGNORECASE)
    patron_neoplasia_gastrica = re.compile(r'neoplasia.*g[aá]strica', re.IGNORECASE)
    patron_tumor_estomago = re.compile(r'tumor.*est[oó]mago', re.IGNORECASE)
    patron_cancer_estomago = re.compile(r'c[aá]ncer.*est[oó]mago', re.IGNORECASE)
    patron_adenocarcinoma_intestinal = re.compile(r'adenocarcinoma.*intestinal', re.IGNORECASE)
    patron_adenocarcinoma_difuso = re.compile(r'adenocarcinoma.*difuso', re.IGNORECASE)
    patron_carcinoma_anillo_sello = re.compile(r'carcinoma.*anillo.*sello', re.IGNORECASE)
    patron_cancer_gastrico_metastasico = re.compile(r'c[aá]ncer.*g[aá]strico.*metast[aá]sico', re.IGNORECASE)
    patron_tumor_gist_gastrico = re.compile(r'tumor.*gist.*g[aá]strico', re.IGNORECASE)

    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_adenocarcinoma_gastrico, na=False) |
        x.str.contains(patron_carcinoma_gastrico, na=False) |
        x.str.contains(patron_linfoma_gastrico, na=False) |
        x.str.contains(patron_sarcoma_gastrico, na=False) |
        x.str.contains(patron_neoplasia_gastrica, na=False) |
        x.str.contains(patron_tumor_estomago, na=False) |
        x.str.contains(patron_cancer_estomago, na=False) |
        x.str.contains(patron_adenocarcinoma_intestinal, na=False) |
        x.str.contains(patron_adenocarcinoma_difuso, na=False) |
        x.str.contains(patron_carcinoma_anillo_sello, na=False) |
        x.str.contains(patron_cancer_gastrico_metastasico, na=False) |
        x.str.contains(patron_tumor_gist_gastrico, na=False)).any(axis=1)
    
    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    return dataframe[(filtro_codigos | filtro_diagnosticos)].copy()

def filtro_dataframe_ca_pulmon_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-10 asociados al cáncer de pulmón
    patron_cancer_pulmon = r'^C34.*'
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de pulmón
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_pulmon, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de pulmón en texto libre
    patron_diagnostico_pulmon = re.compile(r'(\bc[aá]|\bc)([ .]+)?(\bpulm[oó]n|\bpulmonar)', re.IGNORECASE)
    patron_tumor_pulmon = re.compile(r'(\bt|\btumor)([ .]+)?(\bpulm[oó]n|\bpulmonar)', re.IGNORECASE)
    patron_carcinoma_pulmon = re.compile(r'(\bc|\bcarcinoma)([ .]+)?(\bpulm[oó]n|\bpulmonar)', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de pulmón en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_pulmon, na=False) |
        x.str.contains(patron_tumor_pulmon, na=False) |
        x.str.contains(patron_carcinoma_pulmon, na=False)).any(axis=1)


    # Filtrar por la comuna La Florida
    #filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_codigos | filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_prostata_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-! asociados al cáncer de próstata
    patron_cancer_prostata = r'^C61.*'
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de próstata
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_prostata, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de próstata en texto libre
    patron_diagnostico_prostata_1 = re.compile(r'\bc(?!o)[a-zá-ú]*[ \.]*pr[oó]stat[aá][a-zá-ú]*\b', re.IGNORECASE)
    patron_diagnostico_prostata_2 = re.compile(r'\bc[aá][ncer]?.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_3 = re.compile(r'\btumor.*maligno.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_4 = re.compile(r'\bcarci.*noma.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_5 = re.compile(r'\bneoplasia.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_6 = re.compile(r'\badenocarcinoma.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_7 = re.compile(r'\btumor.*prostatic[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_8 = re.compile(r'\bcarci.*noma.*glandula.*prostatic[aá][a-zá-ú]*', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de próstata en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_prostata_1, na=False) |
        x.str.contains(patron_diagnostico_prostata_2, na=False) |
        x.str.contains(patron_diagnostico_prostata_3, na=False) |
        x.str.contains(patron_diagnostico_prostata_4, na=False) |
        x.str.contains(patron_diagnostico_prostata_5, na=False) |
        x.str.contains(patron_diagnostico_prostata_6, na=False) |
        x.str.contains(patron_diagnostico_prostata_7, na=False) |
        x.str.contains(patron_diagnostico_prostata_8, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    #filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_codigos | filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_renal_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-10 asociados al cáncer renal
    patron_cancer_renal = r'^C64.*'

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer renal
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_renal, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer renal en texto libre
    patron_diagnostico_renal = re.compile(r'\bc[aá]*[ \.]*renal.*\b', re.IGNORECASE)
    patron_tumor_renal = re.compile(r'\btu[mor]?.*renal', re.IGNORECASE)
    patron_carcinoma_renal = re.compile(r'\bcarcinoma.*renal', re.IGNORECASE)
    patron_adenocarcinoma_renal = re.compile(r'\badenocarcinoma.*renal', re.IGNORECASE)
    patron_celulas_claras_renal = re.compile(r'\bcélulas claras.*renal', re.IGNORECASE)
    patron_celulas_renales_renal = re.compile(r'\bcélulas renales.*renal', re.IGNORECASE)
    patron_carcinoma_papilar_renal = re.compile(r'\bcarcinoma papilar.*renal', re.IGNORECASE)
    patron_transicionales_renal = re.compile(r'\bcélulas transicionales.*renal', re.IGNORECASE)
    patron_cromofobas_renal = re.compile(r'\bcélulas cromófobas.*renal', re.IGNORECASE)
    patron_medular_renal = re.compile(r'\bcarcinoma medular.*renal', re.IGNORECASE)
    patron_sarcomatoide_renal = re.compile(r'\bcarcinoma sarcomatoide.*renal', re.IGNORECASE)
    patron_granulares_renal = re.compile(r'\btumor.*renal.*células granulares', re.IGNORECASE)
    patron_wilms_renal = re.compile(r'\btumor de Wilms', re.IGNORECASE)
    patron_hipernefroma = re.compile(r'\bhipernefroma', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer renal en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_renal, na=False) |
        x.str.contains(patron_tumor_renal, na=False) |
        x.str.contains(patron_carcinoma_renal, na=False) |
        x.str.contains(patron_adenocarcinoma_renal, na=False) |
        x.str.contains(patron_celulas_claras_renal, na=False) |
        x.str.contains(patron_celulas_renales_renal, na=False) |
        x.str.contains(patron_carcinoma_papilar_renal, na=False) |
        x.str.contains(patron_transicionales_renal, na=False) |
        x.str.contains(patron_cromofobas_renal, na=False) |
        x.str.contains(patron_medular_renal, na=False) |
        x.str.contains(patron_sarcomatoide_renal, na=False) |
        x.str.contains(patron_granulares_renal, na=False) |
        x.str.contains(patron_hipernefroma, na=False) |
        x.str.contains(patron_wilms_renal, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    #filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_codigos | filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_testiculo_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-! asociados al cáncer de testículo
    patron_cancer_testiculo = r'^C62.*'

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de testículo
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_testiculo, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de testículo en texto libre
    patron_diagnostico_testiculo = re.compile(r'\bc[ aá][ncer]?.*(test[ií]culo|escrot[oa]|orqui[td]ectom[ií]a)\b', re.IGNORECASE)
    patron_tumor_testicular = re.compile(r'\btumor.*test[ií]culo\b', re.IGNORECASE)
    patron_neoplasia_testicular = re.compile(r'\bneoplasia.*test[ií]culo\b', re.IGNORECASE)
    patron_carcinoma_embrionario = re.compile(r'\bcarcinoma.*embrionario\b', re.IGNORECASE)
    patron_seminoma = re.compile(r'\bseminoma\b', re.IGNORECASE)
    patron_teratoma_testicular = re.compile(r'\bteratoma.*test[ií]culo\b', re.IGNORECASE)
    patron_metastasis_testicular = re.compile(r'\bmet[aá]stasis.*test[ií]culo\b', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen los diagnósticos de cáncer de testículo en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_testiculo, na=False) |
        x.str.contains(patron_tumor_testicular, na=False) |
        x.str.contains(patron_neoplasia_testicular, na=False) |
        x.str.contains(patron_carcinoma_embrionario, na=False) |
        x.str.contains(patron_seminoma, na=False) |
        x.str.contains(patron_teratoma_testicular, na=False) |
        x.str.contains(patron_metastasis_testicular, na=False)).any(axis=1)


    # Filtrar por la comuna La Florida
    #filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_codigos | filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_leucemia_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Códigos CIE-10 para cánceres hematológicos
    patron_leucemia = r'^C91.*|C92.*|C93.*|C94.*|C95.*'  # Leucemia

    # Filtrar el DataFrame para encontrar las filas que contienen los códigos de cáncer hematológico
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_leucemia, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer hematológico en texto libre
    patron_leucemia_diagnostico = re.compile(r'\bleucemia', re.IGNORECASE)

    #patron_cancer_hematologico_general = re.compile(r'(\bcancer|\bhema?toncologico)', re.IGNORECASE)
    patron_cancer_sangre = re.compile(r'\bcancer.*sangre', re.IGNORECASE)
   
    patron_cancer_sang = re.compile(r'\bcanc.*sang', re.IGNORECASE)
    patron_cancer_med = re.compile(r'\bcanc.*med', re.IGNORECASE)


    # Patrones para identificar neoplasias hematológicas
    patron_neoplasia_sangre = re.compile(r'\bneoplasia.*sangre', re.IGNORECASE)
    patron_neoplasia_medula = re.compile(r'\bneoplasia.*medula', re.IGNORECASE)


    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer hematológico en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
    x.str.contains(patron_leucemia_diagnostico, na=False) |
    #x.str.contains(patron_cancer_hematologico_general, na=False) |
    x.str.contains(patron_cancer_sangre, na=False) |
    x.str.contains(patron_neoplasia_sangre, na=False) |
    x.str.contains(patron_neoplasia_medula, na=False) |
    x.str.contains(patron_cancer_sang, na=False) |
    x.str.contains(patron_cancer_med, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado

def filtro_dataframe_ca_linfoma_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Códigos CIE-10 para cánceres hematológicos
    patron_linfoma = r'^C81.*|C82.*|C83.*|C84.*|C85.*'     # Linfoma


    # Filtrar el DataFrame para encontrar las filas que contienen los códigos de cáncer hematológico
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_linfoma, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer hematológico en texto libre
    patron_linfoma_diagnostico = re.compile(r'\blinfoma', re.IGNORECASE)
    #patron_cancer_hematologico_general = re.compile(r'(\bcancer|\bhema?toncologico)', re.IGNORECASE)
    patron_cancer_linfatico = re.compile(r'\bcancer.*linfatico', re.IGNORECASE)

   

    patron_cancer_linf = re.compile(r'\bcanc.*linf', re.IGNORECASE)


    patron_neoplasia_linfatico = re.compile(r'\bneoplasia.*linfatico', re.IGNORECASE)


    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer hematológico en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
    x.str.contains(patron_linfoma_diagnostico, na=False) |
    x.str.contains(patron_cancer_linfatico, na=False) |
    x.str.contains(patron_neoplasia_linfatico, na=False) |

    x.str.contains(patron_cancer_linf, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado

def filtro_dataframe_ca_mieloma_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Códigos CIE-10 para cánceres hematológicos
    patron_mieloma = r'^C90.*'                             # Mieloma múltiple

    # Filtrar el DataFrame para encontrar las filas que contienen los códigos de cáncer hematológico
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_mieloma, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer hematológico en texto libre
    patron_mieloma_diagnostico = re.compile(r'\bmieloma', re.IGNORECASE)
    patron_cancer_plasma = re.compile(r'\bcancer.*plasma', re.IGNORECASE)
    patron_cancer_plasm = re.compile(r'\bcanc.*plasm', re.IGNORECASE)
    # Patrones para identificar neoplasias hematológicas
    patron_neoplasia_plasma = re.compile(r'\bneoplasia.*plasma', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer hematológico en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
    x.str.contains(patron_mieloma_diagnostico, na=False) |
    #x.str.contains(patron_cancer_hematologico_general, na=False) |
    x.str.contains(patron_cancer_plasma, na=False) |
    x.str.contains(patron_neoplasia_plasma, na=False) |
    x.str.contains(patron_cancer_plasm, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado

def filtro_dataframe_ca_vesical_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Patrón regex para identificar los códigos CIE-10 asociados al cáncer vesical
    patron_cancer_vesical = r'^C67.*'  # Códigos CIE-10 para cáncer de vejiga

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer vesical
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_vesical, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer vesical en texto libre
    patron_diagnostico_vesical = re.compile(r'\bc[aá]*[ \.]*ves[ií]cal*\b', re.IGNORECASE)
    patron_carcinoma_vesical = re.compile(r'\bcarcinoma.*ves[ií]cal[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno_vesical = re.compile(r'\btumor.*maligno.*vejiga|\btumor.*maligno.*ves[ií]cal*', re.IGNORECASE)
    patron_carcinoma_transicional_vesical = re.compile(r'\bcarcinoma.*transicional.*vejiga|\bcarcinoma.*transicional.*ves[ií]cal[a-zá-ú]*', re.IGNORECASE)
    patron_carcinoma_urotelial_vesical = re.compile(r'\bcarcinoma.*urotelial.*vejiga|\bcarcinoma.*urotelial.*ves[ií]cal[a-zá-ú]*', re.IGNORECASE)
    patron_carcinoma_escamoso_vesical = re.compile(r'c\barcinoma.*escamoso.*vejiga|\bcarcinoma.*escamoso.*ves[ií]cal[a-zá-ú]*', re.IGNORECASE)
    patron_adenocarcinoma_vesical = re.compile(r'\badenocarcinoma.*vejiga|\badenocarcinoma.*ves[ií]cal*', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer vesical en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_vesical, na=False) |
        x.str.contains(patron_carcinoma_vesical, na=False) |
        x.str.contains(patron_tumor_maligno_vesical, na=False) |
        x.str.contains(patron_carcinoma_transicional_vesical, na=False) |
        x.str.contains(patron_carcinoma_urotelial_vesical, na=False) |
        x.str.contains(patron_carcinoma_escamoso_vesical, na=False) |
        x.str.contains(patron_adenocarcinoma_vesical, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado



def agregar_dv(resultado_filtrado):
    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    #condicion = (resultado_filtrado['TIPO_IDENTIFICADOR_PACIENTE'] == 'RUN')
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE
    resultado_filtrado['DV'] = resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'].str[-1]

    #resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
    resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'].str[:-1]

    return resultado_filtrado

def agregar_dv_urgencias(resultado_filtrado):

    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE
    resultado_filtrado['DV'] = resultado_filtrado['RUT'].str[-1]

    resultado_filtrado['RUT'] = resultado_filtrado['RUT'].str[:-1]

    return resultado_filtrado

def agregar_dv_inter(resultado_filtrado):
    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE

    resultado_filtrado['DV'] = resultado_filtrado['NRO_IDENTIFICADOR'].str[-1]

    resultado_filtrado['NRO_IDENTIFICADOR'] = resultado_filtrado['NRO_IDENTIFICADOR'].str[:-1]
    resultado_filtrado['NRO_IDENTIFICADOR'] = resultado_filtrado['NRO_IDENTIFICADOR'].astype(str)
    resultado_filtrado['NRO_IDENTIFICADOR'] = resultado_filtrado['NRO_IDENTIFICADOR'].str.replace('.','', regex = False)
    resultado_filtrado['NRO_IDENTIFICADOR'] = resultado_filtrado['NRO_IDENTIFICADOR'].str.replace('-','', regex = False)
    return resultado_filtrado

def obtener_diagnosticos_y_codigos(df, nro_identificacion):
    df_paciente = df[df['NRO_IDENTIFICADOR_PACIENTE'] == nro_identificacion]
    diagnosticos = df_paciente[['DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3', 'DIAGNOSTICO4',
                                'DIAGNOSTICO5', 'DIAGNOSTICO6', 'DIAGNOSTICO7','DIAGNOSTICO8',
                                'DIAGNOSTICO9','DIAGNOSTICO10']].values.flatten()
    codigos_diagnostico = df_paciente[['COD_DIAGNOSTICO1', 'COD_DIAGNOSTICO2', 'COD_DIAGNOSTICO3', 'COD_DIAGNOSTICO4',
                                       'COD_DIAGNOSTICO5', 'COD_DIAGNOSTICO6', 'COD_DIAGNOSTICO7','COD_DIAGNOSTICO8',
                                       'COD_DIAGNOSTICO9','COD_DIAGNOSTICO10']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def obtener_diagnosticos_inter(df, nro_identificacion):
    df_paciente = df[df['NRO_IDENTIFICADOR'] == nro_identificacion]
    diagnosticos = df_paciente[['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']].values.flatten()
    codigos_diagnostico = df_paciente[['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def agregar_diagnosticos_y_codigos_inter(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('NRO_IDENTIFICADOR').agg({'NRO_IDENTIFICADOR': 'first', 'SOSP_DIAG_DESC_1': 'first',
                                                            'SOSP_DIAG_DESC_2': 'first','SOSP_DIAG_DESC_3': 'first','SOSP_DIAG_DESC_4': 'first',
                                                            'SOSP_DIAG_DESC_5': 'first','SOSP_DIAG_COD_1': 'first','SOSP_DIAG_COD_2': 'first',
                                                                      'SOSP_DIAG_COD_3': 'first',
                                                            'SOSP_DIAG_COD_4': 'first', 'SOSP_DIAG_COD_5': 'first'
                                                            }).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['NRO_IDENTIFICADOR'].apply(lambda x: obtener_diagnosticos_inter(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'NRO_IDENTIFICADOR': 'RUT'})
    return resultados

def crear_dataframe_inter(resultado_filtrado, pro_salud):
    nuevo_dataframe = pd.DataFrame({
        'Problema Oncologico': pro_salud,
        'RUT': resultado_filtrado['NRO_IDENTIFICADOR'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOM_PAC'] + ' ' + resultado_filtrado['APELLPAT_PAC']+' '+resultado_filtrado['APELLMAT_PAC'],
        'N° Telefono 1': resultado_filtrado['FONO_MOVIL'],
        'N° Telefono 2': resultado_filtrado['FONO_FIJO_1'],
        'N° Telefono 3': resultado_filtrado['FONO_FIJO_2'],
        'Fecha de SIC': resultado_filtrado['FECHA_SOLIC'],
        'Establecimiento de Origen': resultado_filtrado['ESTAB_ORIGEN_DESC'],
        'Especialidad Destino': resultado_filtrado['ESPECIALIDAD_DESTINO'],
        'Estado de SIC': resultado_filtrado['ESTADO_SOL'],
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def safe_convert_to_int(value):
    try:
        return int(value)
    except ValueError:
        return value

def merge_ambulatorio(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)


    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.
    
    return merged_df 

def merge_urgencia(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.

    return merged_df

def merge_inter(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.

    return merged_df

def filtrar_columnas_relevantes(merged_df):
    nuevo_dataframe = merged_df#.drop_duplicates(subset='RUT', keep='first')
    nuevo_dataframe = nuevo_dataframe[['Problema Oncologico','RUT' ,'DV',
       'Nombre del Paciente', 'N° Telefono 1', 'N° Telefono 2',
       'N° Telefono 3', 'Fecha de SIC','Establecimiento de Origen','Especialidad Destino','Estado de SIC','Fecha Proximo Hito', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO',
       'Nombre agenda Medico especialista']]
    nuevo_dataframe = nuevo_dataframe.rename(columns={'Problema de Salud': 'Problema Oncologico'})
    nuevo_dataframe['Nombre garantia'] = np.nan
    nuevo_dataframe= nuevo_dataframe[['Problema Oncologico',
    'RUT',
    'DV',
    'Nombre del Paciente',
    'N° Telefono 1',
    'N° Telefono 2',
    'N° Telefono 3',
    'Fecha de SIC',
    'Establecimiento de Origen',
    'Especialidad Destino',
    'Estado de SIC',
    'Nombre garantia',
    'Fecha Proximo Hito',
    'DIAGNOSTICOS',
    'CODIGOS_DIAGNOSTICO',
    'Nombre agenda Medico especialista']]
    return nuevo_dataframe

def crear_dataframe_ges(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'Problema Oncologico': dataframe_tiroides['Problema de salud'],
        'RUT': dataframe_tiroides['RUT'],
        'DV': dataframe_tiroides['DV'],
        'Nombre del Paciente':dataframe_tiroides['Nombre'],
        'N° Telefono 1': dataframe_tiroides['Telefonos.x'],
        'N° Telefono 2': dataframe_tiroides['Telefonos.y'],
        'Fecha de SIC': dataframe_tiroides['Fecha inicio'],
        'Establecimiento de Origen': np.nan,
        'Especialidad Destino': np.nan,
        'Estado de SIC': np.nan,
        'N° Telefono 3': np.nan,
        'Nombre garantia': dataframe_tiroides['Nombre garantia'],
        'Fecha Proximo Hito': np.nan,
        'DIAGNOSTICOS': np.nan,
        'CODIGOS_DIAGNOSTICO': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })
    return nuevo_dataframe_2

def crear_dataframe_ultima_atencion(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'RUT': dataframe_tiroides['RUT'],
        'Fecha Ultima atencion': dataframe_tiroides['HORA_TERMINO_ATENCION'],
        'Profesional Ultima atencion': dataframe_tiroides['ESTAMENTO_PROFESIONAL_ATENCION']+ ' '+
        dataframe_tiroides['NOMBRE_PROFESIONAL_ATENCION']+ ' ' + dataframe_tiroides['DESCRIPCION'] + ' '+
        dataframe_tiroides['ESPECIALIDAD_AGENDA']
    })
    return nuevo_dataframe_2

def gestion_fecha(dataframe):
    fecha_hoy = datetime.today()
    fecha_inicio = fecha_hoy - timedelta(days=10)
    fecha_inicio = pd.to_datetime(fecha_inicio, format='%d/%m/%Y')
    #fecha_inicio = pd.to_datetime('01/01/2023', format='%d/%m/%Y')
    # Filtrar el DataFrame


    return dataframe[dataframe['FECHA_SOLIC'] >= fecha_inicio]

regex_esp = re.compile(r'(?i)\bbroncopulmonar|cirug[íi]a.*(abdominal|adulto|cabeza.*cuello|t[óo]rax|digestiva|general)|coloproctolog[íi]a|dermatolog[íi]a|endocrinolog[íi]a|gastroenterolog[íi]a|hematolog[íi]a|medicina.*interna|traumatolog[íi]a|urolog[íi]a', re.IGNORECASE)

def proceso_dataframe_inter(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada', 'Rechazada'])]
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    
    dataframe = gestion_fecha(dataframe)

    # Definir la fecha de inicio para el filtro
    resultado_filtrado = filtro_dataframe_ca_tiroides_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado, 'Cancer de Tiroides')
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe


def proceso_dataframe_inter_colorectal(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada', 'Rechazada'])]
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    # Definir la fecha de inicio para el filtro
    dataframe = gestion_fecha(dataframe)

    # Definir la fecha de inicio para el filtro
    resultado_filtrado = filtro_dataframe_ca_colorectal_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado, 'Cancer Colorectal')
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def proceso_dataframe_inter_gastrico(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada', 'Rechazada'])]
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    # Definir la fecha de inicio para el filtro
    dataframe = gestion_fecha(dataframe)
    # Definir la fecha de inicio para el filtro
    resultado_filtrado = filtro_dataframe_ca_gastrico_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado, 'Cancer Gástrico')
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def proceso_dataframe_inter_pulmon(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada', 'Rechazada'])]
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    dataframe = gestion_fecha(dataframe)

    # Definir la fecha de inicio para el filtro
    resultado_filtrado = filtro_dataframe_ca_pulmon_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado, 'Cancer Pulmón')
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def proceso_dataframe_inter_renal(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada', 'Rechazada'])]
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    # Definir la fecha de inicio para el filtro
    dataframe = gestion_fecha(dataframe)
    # Definir la fecha de inicio para el filtro
    resultado_filtrado = filtro_dataframe_ca_renal_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado, 'Cancer Renal')
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def proceso_dataframe_inter_prostata(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada', 'Rechazada'])]
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    # Definir la fecha de inicio para el filtro
    dataframe = gestion_fecha(dataframe)
    # Definir la fecha de inicio para el filtro
    resultado_filtrado = filtro_dataframe_ca_prostata_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado, 'Cancer Prostata')
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def proceso_dataframe_inter_testiculo(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada', 'Rechazada'])]
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    # Definir la fecha de inicio para el filtro
    dataframe = gestion_fecha(dataframe)
    # Definir la fecha de inicio para el filtro
    resultado_filtrado = filtro_dataframe_ca_testiculo_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado, 'Cancer Testículo')
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def proceso_dataframe_inter_leucemia(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada', 'Rechazada'])]
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    # Definir la fecha de inicio para el filtro
    dataframe = gestion_fecha(dataframe)
    # Definir la fecha de inicio para el filtro
    resultado_filtrado = filtro_dataframe_ca_leucemia_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado, 'Leucemia')
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def proceso_dataframe_inter_linfoma(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada', 'Rechazada'])]
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    # Definir la fecha de inicio para el filtro
    dataframe = gestion_fecha(dataframe)
    # Definir la fecha de inicio para el filtro
    resultado_filtrado = filtro_dataframe_ca_linfoma_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado, 'Linfoma')
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def proceso_dataframe_inter_mieloma(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada', 'Rechazada'])]
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    # Definir la fecha de inicio para el filtro
    dataframe = gestion_fecha(dataframe)
    # Definir la fecha de inicio para el filtro
    resultado_filtrado = filtro_dataframe_ca_mieloma_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado, 'Mieloma')
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def proceso_dataframe_inter_vesical(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada', 'Rechazada'])]
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    # Definir la fecha de inicio para el filtro
    dataframe = gestion_fecha(dataframe)
    # Definir la fecha de inicio para el filtro
    resultado_filtrado = filtro_dataframe_ca_vesical_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado, 'Mieloma')
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def obtener_fechas_amb(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
        dataframe['DV'] = dataframe['NRO_IDENTIFICADOR_PACIENTE'].str[-1]
        dataframe['RUT'] = dataframe['NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
        



        resultados_totales.append(dataframe)
        
    dataframe = pd.concat(resultados_totales)

    
    # Filtrar por las especialidades de Endocrinología Adulto y Endocrinología Infantil
    endocrinologia_df = dataframe[dataframe['ESPECIALIDAD_AGENDA'].isin(['Endocrinología Adulto','Cirugia Cabeza y Cuello'])]
    endocrinologia_df = endocrinologia_df[endocrinologia_df['ESTAMENTO_PROFESIONAL_ATENCION']=='Médico']
    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
    endocrinologia_df = endocrinologia_df[~endocrinologia_df['ESTADO_CITA'].isin(['No realizada', 'Anulada'])]
    # Convertir la columna 'HORA_TERMINO_ATENCION' a formato de fecha y hora
    endocrinologia_df['HORA_TERMINO_ATENCION'] = pd.to_datetime(endocrinologia_df['HORA_TERMINO_ATENCION'], format='mixed')


    # Agrupar por RUT y obtener la última fecha de la hora de atención para cada grupo
    #ultima_fecha_por_rut = endocrinologia_df.groupby('RUT').agg({'HORA_TERMINO_ATENCION': 'max'}).reset_index()

    ultima_fecha_por_rut =  endocrinologia_df.groupby('RUT').agg({
    'HORA_TERMINO_ATENCION': 'max',
    'NOMBRE_PROFESIONAL_ATENCION': 'first',
    'ESTAMENTO_PROFESIONAL_ATENCION': 'first',
    'DESCRIPCION': 'first',
    'ESPECIALIDAD_AGENDA': 'first'
}).reset_index()
    # Fusionar el DataFrame original con el DataFrame resultante para recuperar NOMBRE_PROFESIONAL_ATENCION y ESTAMENTO_PROFESIONAL_ATENCION
    resultado = ultima_fecha_por_rut.merge(endocrinologia_df[['RUT', 'HORA_TERMINO_ATENCION', 'NOMBRE_PROFESIONAL_ATENCION', 'ESTAMENTO_PROFESIONAL_ATENCION','DESCRIPCION']], on=['RUT', 'HORA_TERMINO_ATENCION'], how='left')

    return ultima_fecha_por_rut 

In [70]:
# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)
# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)

# Ruta archivos pc HLF

In [71]:
dir_actual = os.getcwd()
dir_archivo_csv = os.path.join(dir_actual,'..','Reporteria.lnk')

In [72]:
import win32com.client

def get_target_path(lnk_file):
    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(lnk_file)
    return shortcut.Targetpath

# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Reporteria.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_reporteria = get_target_path(ruta_del_lnk)

#print("Ruta de destino:", ruta_reporteria)


In [73]:
# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Unidad_Gestion_Del_Paciente.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_ges = get_target_path(ruta_del_lnk)

#print("Ruta de destino:", ruta_ges)

In [74]:
#Reportes SIC

patron_sic = os.path.join(ruta_reporteria,'Reportes_Solicitudes','Mensual', '*Informe_Solicitudes_DerivacionCE*')

archivos_sic  = glob.glob(patron_sic)

archivos_ordenados_sic  = sorted(archivos_sic , key=os.path.getmtime, reverse = True)

rutas_archivos_4 = archivos_ordenados_sic[:1]

rutas_archivos_4 = rutas_archivos_4[0]

# Termina ruta archivos HLF

In [75]:
#dir_actual = os.getcwd()
# Construir la ruta relativa al directorio donde se encuentra el archivo CSV
#dir_archivo_csv = os.path.join(dir_actual, '..','..')
#dir_datos_hlf = os.path.join(dir_archivo_csv, 'Datos HLF')

In [76]:
#rutas_archivos_4 = os.path.join(dir_datos_hlf, '20240603_Informe_Solicitudes_DerivacionCE.csv')

In [77]:

resultado_final_4 = proceso_dataframe_inter(rutas_archivos_4)

In [78]:
resultado_final_5 = proceso_dataframe_inter_colorectal(rutas_archivos_4)

In [79]:
resultado_final_6 = proceso_dataframe_inter_gastrico(rutas_archivos_4)

In [80]:
resultado_final_7 = proceso_dataframe_inter_prostata(rutas_archivos_4)

In [81]:
resultado_final_8 = proceso_dataframe_inter_renal(rutas_archivos_4)

In [82]:
resultado_final_9 = proceso_dataframe_inter_pulmon(rutas_archivos_4)

In [83]:
resultado_final_10 = proceso_dataframe_inter_testiculo(rutas_archivos_4)

In [84]:
resultado_final_11 = proceso_dataframe_inter_leucemia(rutas_archivos_4)

In [85]:
resultado_final_12 = proceso_dataframe_inter_linfoma(rutas_archivos_4)

In [86]:
resultado_final_13 = proceso_dataframe_inter_mieloma(rutas_archivos_4)

In [87]:
resultado_final_14 = proceso_dataframe_inter_vesical(rutas_archivos_4)

In [88]:
resultado_final_14

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
0,Mieloma,4467192,1,Jaime Harold Fernandez Perez,78560046,93860881,NaN,2024-06-17,CESFAM LA FLORIDA,Urología Adulto,Solicitada,NaN,NaN,"[Lesión de sitios contiguos de la vejiga, Lesi...",[C67.8],NaN
1,Mieloma,5036049,0,HÉCTOR HUGO SILVA VERA,998259384,9976098046,NaN,2024-06-17,HLF,Cirugía Adulto,Pendiente de Autorizar,NaN,NaN,"[Tumor maligno de la vejiga urinaria, CA VESICAL]",[C67],NaN


# Generar dataframes

## Dataframe GES HLF

In [42]:

#os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')
ruta_del_archivo = os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')


# Carga el archivo XLSX en un DataFrame
dataframe = pd.read_excel(ruta_del_archivo)

## Dataframe GES local

In [43]:
#ruta_del_archivo = os.path.join(dir_datos_hlf, 'V5_Cruce.xlsx')


# Carga el archivo XLSX en un DataFrame
#dataframe = pd.read_excel(ruta_del_archivo)

In [44]:
dataframe['Nombre garantia'].value_counts()

Nombre garantia
Confirmación Diagnóstica                          2485
Confirmación-Diagnóstica                           709
Tratamiento Quirúrgico con AV <= 0,3               355
Tratamiento Quirúrgico en 2° Ojo                   219
Tratamiento Quirúrgico con AV <= 0,1               213
                                                  ... 
Seguimiento Primer Control                           1
Tratamiento Retención Urinaria Aguda Repetida.       1
Etapificación Unilateral Derecha                     1
Consulta con Especialista                            1
Tratamiento Quirúrgico Endoprótesis Izquierda        1
Name: count, Length: 72, dtype: int64

In [45]:
dataframe['Nombre garantia'].unique()

array(['Seguimiento', 'Diagnóstico', 'Consulta Médica',
       'Confirmación-Diagnóstica',
       'Confirmación Diagnóstica Leucemia Crónica',
       'Tratamiento Quirúrgico Alto Riesgo', 'Confirmación Diagnóstica',
       'Prevención Secundaria IAM', 'Confirmación',
       'Tratamiento Quirúrgico con AV <= 0,1',
       'Tratamiento Quirúrgico con AV <= 0,3',
       'Tratamiento Pre-Invasor Alto Grado',
       'Atención por Especialista Diabetes',
       'Confirmación Mama Izquierda', 'Confirmación Mama Derecha',
       'Tratamiento Quirúrgico en 2° Ojo', 'Tratamiento Paliativo',
       'Tratamiento Primario Mama Derecha', 'Atención Especialista',
       'Etapificación', 'Diagnóstico-Etapificación Mama Derecha.',
       'Re-estadificación de Riesgo', 'Consulta con Especialista',
       'Rehabilitación - Andador con cuatro ruedas y canasta',
       'Tratamiento Quirúrgico',
       'Atención por Especialista Artrosis de Rodilla Leve o Moderada',
       'Acceso Vascular para Hemodiálisis'

In [46]:
dataframe

,Problema de salud,RUT,DV,Nombre,Fecha inicio,Fecha limite,Nombre garantia,Telefonos.x,DIRECCION_NUMERO,BLOCK_DEPTO,DIRECCION.THALAMUS,Telefonos.y
0,Agresión Sexual Aguda . {Decreto Nro 72/2022},1,1,Cod.Confidencial = 1011246428,2024-06-05,2024-07-10,Seguimiento,#ND,#ND,#ND,#ND,#ND
1,Agresión Sexual Aguda . {Decreto Nro 72/2022},1,1,Cod.Confidencial = 4171996,2024-06-09,2024-07-15,Seguimiento,#ND,#ND,#ND,#ND,#ND
2,Agresión Sexual Aguda . {Decreto Nro 72/2022},1,1,Cod.Confidencial = 2191730,2024-06-17,2024-07-22,Seguimiento,#ND,#ND,#ND,#ND,#ND
3,VIH/SIDA . {decreto nº 1/2010},1,1,Cod.Confidencial = 1007613980,2024-05-31,2024-07-15,Diagnóstico,#ND,#ND,#ND,#ND,#ND
4,VIH/SIDA . {decreto nº 1/2010},1,1,Cod.Confidencial = 1013766344,2024-06-14,2024-07-29,Consulta Médica,#ND,#ND,#ND,#ND,#ND
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,Retinopatía Diabética . {decreto nº 228},9968823,8,"ALEGRÍA HUERTA, MARÍA ALEJANDRINA DEL CARMEN",2024-04-29,2024-07-29,Confirmación Diagnóstica,- / - / 0 / / 78773579 / - / -/78773579//.,PASAJE-El Apero-El Rodeo-1466,-,"Pasaje El Apero 1466,",#ND
5106,Catarátas Proceso de Diagnóstico {decreto nº 228},9968849,1,"ARRIAGADA PINTO, MARCO AURELIO",2023-10-10,2024-04-08,Confirmación-Diagnóstica,- / - / 45100096 / / 45100096 / - / -/.,CALLE-Las Amapolas-Santa Raquel-9648,-,#ND,#ND
5107,Cáncer de Mama Izquierda {decreto nº 228},9974092,2,"MENUTIS LEIVA, ELPIDA DEL CARMEN",2024-04-01,2024-05-16,Diagnóstico-Etapificación Mama Izquierda.,- / - / 68151549 / 63306847 / 68151549 / 68151...,PASAJE-San Jorge-San Jorge-949,-,#ND,#ND
5108,Catarátas Rama Bilateral {decreto nº 228},9984893,6,"MUÑOZ LEÓN, LAURA ROSA",2024-03-06,2024-09-02,Tratamiento Quirúrgico en 2° Ojo,- / - / / / 50492270 / 86434517 / -/99504922...,AVENIDA-el araucano--11529,-,"Calle el araucano 10529,",#ND


In [47]:
#regex_ges = re.compile(r'\b(?:confirmaci[oó]n.*diagn[oó]stica(?! \(sin.*etapificacion\))|confirmaci[oó]n|atenci[óo]n.*de.*especialidad|diagnostico.*consulta.*especialista|evaluaci[oó]n.*por.*especialista|consulta.*especialista)\b')



#regex_ges = pattern = re.compile(r'\bConfirmación Diagnóstica', re.IGNORECASE)
regex_ges = re.compile(r'\bconfirmaci[oó]n diagn[oó]stica(?! y etapificaci[oó]n)|\bconfirmaci[oó]n\b(?! diagn[oó]stica y etapificaci[oó]n)|confirmaci[oó]n [-\w\s]+ diagn[oó]stica\b|atenci[óo]n.*especiali(sta|dad)|(consulta|evaluaci[óo]n|diagn[oó]stico).*especialista', re.IGNORECASE)
#regex_ges = re.compile(r'atenci[óo]n.*especiali(sta|dad)', re.IGNORECASE)
#regex_ges = re.compile(r'(consulta|evaluaci[óo]n|diagn[oó]stico).*especialista', re.IGNORECASE)

dataframe = dataframe[dataframe['Nombre garantia'].str.contains(regex_ges, na=False)]

In [48]:
#confimacion diagnostica - confimacion - atención de especialidad - confimacion diag (sin etapificacion)  - diagnostico consulta especialista - evaluacion por especialista - consulta especialista 

In [49]:
dataframe

,Problema de salud,RUT,DV,Nombre,Fecha inicio,Fecha limite,Nombre garantia,Telefonos.x,DIRECCION_NUMERO,BLOCK_DEPTO,DIRECCION.THALAMUS,Telefonos.y
9,Catarátas Proceso de Diagnóstico {decreto nº 228},10005555,4,"FARÍAS MENESES, AURELIA DE LAS MERCE",2024-01-22,2024-07-22,Confirmación-Diagnóstica,- / - / 77504577 / / 52048581 / / -/52048581//.,CALLE-Psje Berilio-Los Industriales-10649,-,"Avenida Psje Berilio 10649,",#ND
10,Leucemia Adulto Leucemia Crónica {decreto nº 44},10042988,8,"ROSALES GUTIÉRREZ, MARGARITA",2024-04-05,2024-06-04,Confirmación Diagnóstica Leucemia Crónica,- / - / 0 / / 93997626 / 86979838 / -/.,PASAJE-laguna del maule-Santa Maria De La Estr...,-,#ND,#ND
12,Catarátas Proceso de Diagnóstico {decreto nº 228},10051904,6,"QUIJADA SÁNCHEZ, SYLVIA",2024-06-04,2024-12-02,Confirmación-Diagnóstica,#ND,#ND,#ND,#ND,#ND
13,Retinopatía Diabética . {decreto nº 228},10057389,K,"DONAIRE ORTIZ, RAIMUNDO ALEJANDRO",2024-03-16,2024-06-14,Confirmación Diagnóstica,- / - / 46528184 / 95718980 / 941366250 / 7674...,PASAJE-LAS ROSAS-La Higuera-3986,-,#ND,FIJO: 63560869 MOVIL: 76742162
15,Cáncer Colorectal . {decreto n° 4/2013},10069096,9,"ROMERO VALENZUELA, MARGARITA ISABEL DEL CARMEN",2024-04-30,2024-06-14,Confirmación,- / - / 44775429 / / 44775429 / 44775429 / -/.,PASAJE-mercurio--330,-,#ND,#ND
...,...,...,...,...,...,...,...,...,...,...,...,...
5100,Catarátas Proceso de Diagnóstico {decreto nº 228},9879375,5,"VARGAS PÉREZ, JORGE RUBÉN",2023-11-22,2024-05-20,Confirmación-Diagnóstica,- / - / NOINF / / 56929313 / / -/.,NINGUNO-ANTARTICA--2440,-,#ND,#ND
5103,Vicios de Refracción Sospecha Vicios de Refrac...,9911750,8,"CANIO REYES, ANTONIO",2024-04-22,2024-10-21,Confirmación Diagnóstica,- / - / 94956250 / 0 / 54924512 / 37412934 / 2...,PASAJE-Aillaguriche-San Esteban-1397,-,"Pasaje Aillaguriche 1397,",#ND
5104,Retinopatía Diabética . {decreto nº 228},9935961,7,"NAVARRO PAREDES, MARÍA DE LAS MERCEDES",2024-04-15,2024-07-15,Confirmación Diagnóstica,- / - / 28661981 / 82695382 / 82695382 / 27043...,CALLE-VIÑA DEL MAR-Alberto Larraguibel-6466,-31 A,"Calle VIÑA DEL MAR 6466, DP 31A",#ND
5105,Retinopatía Diabética . {decreto nº 228},9968823,8,"ALEGRÍA HUERTA, MARÍA ALEJANDRINA DEL CARMEN",2024-04-29,2024-07-29,Confirmación Diagnóstica,- / - / 0 / / 78773579 / - / -/78773579//.,PASAJE-El Apero-El Rodeo-1466,-,"Pasaje El Apero 1466,",#ND


In [50]:
dataframe['Fecha inicio'] = pd.to_datetime(dataframe['Fecha inicio'], format='%Y-%m-%d')

In [51]:
fecha_hoy = datetime.today()
fecha_inicio = fecha_hoy - timedelta(days=10)

fecha_inicio = pd.to_datetime(fecha_inicio, format='%Y-%m-%d')
    # Filtrar el DataFrame
dataframe = dataframe[dataframe['Fecha inicio'] >= fecha_inicio]

In [52]:
dataframe

,Problema de salud,RUT,DV,Nombre,Fecha inicio,Fecha limite,Nombre garantia,Telefonos.x,DIRECCION_NUMERO,BLOCK_DEPTO,DIRECCION.THALAMUS,Telefonos.y
43,Cáncer de Mama Derecha {decreto nº 228},10385694,9,"TRONCOSO PICÓN, CECILIA PERLA",2024-06-17,2024-08-01,Confirmación Mama Derecha,#ND,#ND,#ND,#ND,#ND
44,Cáncer Colorectal . {decreto n° 4/2013},10387262,6,"STUARDO MELO, ELIZABETH VALERIA",2024-06-18,2024-08-02,Confirmación,- / - / 89304816 / 74142135 / 89304816 / 89304...,CALLE-Condell-Los Rosales-8255,-,"Avenida Condell 8255,",#ND
53,Diabetes Mellitus Tipo 2 . {decreto nº 228},10449299,1,"FUENTES CASTILLO, PATRICIO ROBERTO",2024-06-17,2024-08-16,Atención por Especialista Diabetes,- / - / 0 / / 47914571 / - / -/.,PASAJE-REINA LUISA-Los Copihues-6852,-,#ND,#ND
75,Artritis Reumatoide . {decreto nº 44},10698629,0,"MAYA PIZARRO, GLORIA SUSANA",2024-06-19,2024-09-17,Confirmación Diagnóstica,#ND,#ND,#ND,#ND,#ND
85,Catarátas Proceso de Diagnóstico {decreto nº 228},10826318,0,"HERRERA GUINEO, JUAN CARLOS",2024-06-17,2024-12-16,Confirmación-Diagnóstica,- / - / 85229190 / 85229190 / 85229190 / 85229...,CALLE-Tauro-Patria Nueva-10483,-,"Avenida Tauro 10483,",#ND
...,...,...,...,...,...,...,...,...,...,...,...,...
4800,Catarátas Proceso de Diagnóstico {decreto nº 228},8384545,7,"JARA MONTECINO, LILIAM DEL CARMEN",2024-06-17,2024-12-16,Confirmación-Diagnóstica,- / - / 8818952 / / 81224761 / 8818952 / -/99...,PASAJE-Galicia-España-1834,-,"Pasaje Galicia 1834,",#ND
4894,Vicios de Refracción Sospecha Vicios de Refrac...,8626525,7,"BRAGAZZI PADILLA, ANGELLO LUIS",2024-06-17,2024-12-16,Confirmación Diagnóstica,- / - / 22874155 / / 90903418 / 22874155 / -/.,PASAJE-SANTA JULIA-Pad 17 Vmo-1150-J,-,#ND,#ND
4899,Vicios de Refracción Sospecha Vicios de Refrac...,8654358,3,"SILVA MEDINA, MARIANELA GRACIELA",2024-06-17,2024-12-16,Confirmación Diagnóstica,- / - / 85301352 / / 85301352 / 85301352 / -/...,PASAJE-cinco-Don Francisco-1452,-,"Calle El parron 1452,",FIJO: MOVIL: 09985301352
4961,Catarátas Proceso de Diagnóstico {decreto nº 228},8964371,6,"SANTIBÁÑEZ CARREÑO, SEGUNDO EDUARDO",2024-06-18,2024-12-16,Confirmación-Diagnóstica,- / - / / / 34961549 / - / -/9952390612//.,PASAJE-Millamar--6699,0 -0,"Pasaje Millamar 6699,",FIJO: NO ESPECIFICADO MOVIL: 34961549


In [53]:
#confimacion diagnostica - confimacion - atención de especialidad - confimacion diag (sin etapificacion)  - diagnostico consulta especialista - evaluacion por especialista - consulta especialista 

In [54]:
# Construir la ruta completa al archivo CSV
dataframe['RUT']= dataframe['RUT'].apply(lambda x : str(x))
dataframe['RUT COMPLETO'] = dataframe['RUT'].astype(str)  + dataframe['DV'].astype(str)


In [55]:
dataframe['Fecha inicio'].min()

Timestamp('2024-06-16 00:00:00')

In [56]:
import pandas as pd

# Suponiendo que 'dataframe' es tu DataFrame original
# Utilizamos una expresión regular para buscar las palabras clave 'pulmon', 'colorecta', 'linfoma', y 'leucemia'
regex = r'(?i)\b(pulmón|colorectal|linfoma|leucemia|g[aá]strico)\b'

dataframe_tiroides = dataframe[dataframe['Problema de salud'].str.contains(regex, na=False)]


In [57]:
dataframe_tiroides['Problema de salud'].value_counts()

Problema de salud
Cáncer Colorectal . {decreto n° 4/2013}    4
Name: count, dtype: int64

In [58]:
resultado_final = [resultado_final_4]

In [59]:

dataframe_tiroides["Fecha inicio"]=pd.to_datetime(dataframe_tiroides["Fecha inicio"])
# Ordenar en orden descendente por la columna "FECHA_CITA"
dataframe_tiroides = dataframe_tiroides.sort_values(by='Fecha inicio', ascending=False)


# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"
dataframe_tiroides = dataframe_tiroides.drop_duplicates(subset=['RUT','Problema de salud'], keep='first')

nuevo_dataframe_2 = crear_dataframe_ges(dataframe_tiroides)

resultado_final.append(nuevo_dataframe_2)



#if ultimo_csv:
#    print("La dirección del último archivo .csv en la carpeta es:", ultimo_csv)
    
#    resultado_final.append(proceso_dataframe_ultimo(ultimo_csv))
#    df_concatenated = pd.concat(resultado_final)

#else:
#    print("No se encontraron archivos .csv en la carpeta.")
#    df_concatenated = pd.concat(resultado_final)
 # Concatenar los DataFrames
resultado_final.append(resultado_final_5)
resultado_final.append(resultado_final_6)
resultado_final.append(resultado_final_7)
resultado_final.append(resultado_final_8)
resultado_final.append(resultado_final_9)
resultado_final.append(resultado_final_10)
resultado_final.append(resultado_final_11)
resultado_final.append(resultado_final_12)
resultado_final.append(resultado_final_13)
resultado_final.append(resultado_final_14)
df_concatenated = pd.concat(resultado_final)

      




# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"

#consolidated_df = df_concatenated.groupby('RUT').apply(lambda x: x.fillna(method='ffill').fillna(method='bfill')).drop_duplicates().reset_index(drop=True)

#resultado_filtrado_2 = consolidated_df.drop_duplicates(subset='RUT', keep='first')

resultado_filtrado_2 = df_concatenated#.drop_duplicates(subset='RUT', keep='first')

# Iterar sobre las filas del primer DataFrame
for index, row in resultado_filtrado_2.iterrows():
    rut = row['RUT']
    
    # Buscar el valor correspondiente en el segundo DataFrame
    nombre_garantia = nuevo_dataframe_2.loc[nuevo_dataframe_2['RUT'] == str(rut), 'Nombre garantia'].values
    
    # Encontrar la fila correspondiente en el primer DataFrame y asignar el valor
    resultado_filtrado_2.loc[resultado_filtrado_2['RUT'] == rut, 'Nombre garantia'] = nombre_garantia[0] if nombre_garantia.size > 0 else None

resultado_filtrado_2['RUT'] = resultado_filtrado_2['RUT'].apply(lambda x: str(x))


In [60]:
nombre_archivo = 'Total pacientes_derivaciones.xlsx'  # Nombre del archivo de Excel
#resultado_filtrado_2.to_excel(nombre_archivo, index=False)  # index=False para no incluir el índice en el archivo de Excel


In [61]:
resultado_filtrado_2[(resultado_filtrado_2['Especialidad Destino'].isna())]

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
44,Cáncer Colorectal . {decreto n° 4/2013},10387262,6,"STUARDO MELO, ELIZABETH VALERIA",- / - / 89304816 / 74142135 / 89304816 / 89304...,#ND,NaN,2024-06-18,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
666,Cáncer Colorectal . {decreto n° 4/2013},27591678,1,"SOLIS SANCHEZ, VIRGINIA",#ND,FIJO: 59957998 MOVIL: 59957998,NaN,2024-06-18,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
2205,Cáncer Colorectal . {decreto n° 4/2013},5374042,1,"MEDINA RETAMAL, PEDRO ANSELMO",- / - / 59132362 / 59132362 / 59132362 / 59132...,FIJO: 59132362 MOVIL: 59132362,NaN,2024-06-18,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
3109,Cáncer Colorectal . {decreto n° 4/2013},6218895,2,"ÁLVAREZ CORONADO, IDA BERTA",#ND,#ND,NaN,2024-06-17,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN


In [62]:
resultado_filtrado_2['Problema Oncologico'].value_counts()

Problema Oncologico
Cáncer Colorectal . {decreto n° 4/2013}    4
Cancer Prostata                            4
Cancer de Tiroides                         3
Mieloma                                    2
Cancer Renal                               1
Cancer Testículo                           1
Leucemia                                   1
Linfoma                                    1
Name: count, dtype: int64

In [63]:
resultado_filtrado_2[(resultado_filtrado_2['Especialidad Destino'].isna())]['Problema Oncologico'].value_counts()

Problema Oncologico
Cáncer Colorectal . {decreto n° 4/2013}    4
Name: count, dtype: int64

In [64]:
resultado_filtrado_2[~(resultado_filtrado_2['DIAGNOSTICOS'].isna())]

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
0,Cancer de Tiroides,10675861,1,SERGIO ENRIQUE GUTIÉRREZ REY,56992739749,56954775325,NaN,2024-06-20,HLF,Endocrinología Adulto,Pendiente de Autorizar,None,NaN,[Tumor maligno de la glandula tiroides],[C73],NaN
1,Cancer de Tiroides,15543500,3,ANGELINA DEL CARMEN CORTÉS MARGUIROTT,988290958,NaN,NaN,2024-06-21,HLF,Endocrinología Adulto,Pendiente de Autorizar,None,NaN,[Tumor maligno de la glandula tiroides],[C73],NaN
2,Cancer de Tiroides,21129381,0,KATHERINE ALEJANDRA PALACIOS BARRIGA,56922176425,56945512698,NaN,2024-06-17,HLF,Cirugia Cabeza y Cuello,Pendiente de Autorizar,None,NaN,"[Bocio multinodular no toxico, Tumor maligno d...","[C73, E042]",NaN
0,Cancer Prostata,6811081,5,CARLOS JORGE JIMÉNEZ OGALDE,0991561036,953506582,NaN,2024-06-18,HLF,Medicina Interna,Solicitada,None,NaN,[Tumor maligno de la prostata],[C61],NaN
1,Cancer Prostata,8029404,2,Marco Alberto Soto Pallamares,51583676,51583676,NaN,2024-06-17,CESFAM BELLAVISTA,Urología Adulto,Solicitada,None,NaN,[Cáncer De Próstata En Personas De 15 Años Y M...,[C61],NaN
2,Cancer Prostata,9034836,1,Carlos Soto Espinoza,34632345,NO ESPECIFICADO,NaN,2024-06-17,CESFAM TRINIDAD,Urología Adulto,Solicitada,None,NaN,[Cáncer De Próstata En Personas De 15 Años Y M...,[C61],NaN
3,Cancer Prostata,27227863,6,Carlos Alfonso Garcia Jordan,36868620,NO ESPECIFICADO,NaN,2024-06-17,CESFAM BELLAVISTA,Urología Adulto,Solicitada,None,NaN,[Cáncer De Próstata En Personas De 15 Años Y M...,[C61],NaN
0,Cancer Renal,9092011,1,LUIS FERNANDO MIRANDA FLORES,56973109503,56966223691,NaN,2024-06-17,HLF,Urología Adulto,Solicitada,None,NaN,"[HERNIA INCISIONALES, Diabetes mellitus no ins...","[K833, E11, I10]",NaN
0,Cancer Testículo,18547328,7,Daniel Alejandro Mundaca Carrasco,65787645,65787645,NaN,2024-06-17,CESFAM LOS CASTAÑOS,Urología Adulto,Solicitada,None,NaN,"[Tumor maligno del testículo, ( de los órganos...",[C62],NaN
0,Leucemia,4022868,3,EDUARDO SEGUNDO ZEPEDA CERDA,995063586,56223584826,990928423,2024-06-18,HLF,Hematología Adulto,Pendiente de Autorizar,None,NaN,[Leucemia mieloide aguda],[C920],NaN


In [65]:
resultado_filtrado_2[resultado_filtrado_2['RUT']==16725244]

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista


In [66]:
resultado_filtrado_2[resultado_filtrado_2['RUT']=='8861552']

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista


In [67]:
resultado_filtrado_2['RUT'].value_counts()

RUT
10675861    1
9034836     1
4467192     1
4645275     1
4022868     1
18547328    1
9092011     1
27227863    1
8029404     1
15543500    1
6811081     1
6218895     1
5374042     1
27591678    1
10387262    1
21129381    1
5036049     1
Name: count, dtype: int64

In [68]:
resultado_filtrado_2[resultado_filtrado_2['RUT'] == '8825939']

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista


# DEBUGG

In [224]:
aux = cargar_dataframe_ambulatorio(rutas_archivos_4)

In [184]:
aux

In [225]:
list(aux.columns)

['PAC_ID',
 'TIPO_INDENTIF',
 'NRO_IDENTIFICADOR',
 'NOM_PAC',
 'APELLPAT_PAC',
 'APELLMAT_PAC',
 'FECHA_NAC_PAC',
 'EDAD_PAC',
 'SEXO',
 'NACIONALIDAD_PAC',
 'FONO_MOVIL',
 'FONO_FIJO_1',
 'FONO_FIJO_2',
 'DIRECCION_PAC',
 'COMUNA',
 'TIPO_PREVISION_EPISODIO',
 'TRAMO',
 'PRAIS',
 'ID_SOLICITUD',
 'FECHA_CREACION_SOL',
 'FECHA_SOLIC',
 'FECHA_DESEABLE_AT',
 'TIPO_SOLICITUD',
 'ESTAB_ORIGEN_COD',
 'ESTAB_ORIGEN_DESC',
 'ESTAB_ORIGEN_LINEA_ASIS',
 'ESPECIALIDAD_ORIGEN',
 'RUN_PROF_SOLICITA',
 'DV_PROF_SOLICITA',
 'PROF_SOLICITANTE',
 'PRIORIDAD_ORIGEN',
 'NRO_GES',
 'DESC_GES',
 'TIPO_DIAG_1',
 'SOSP_DIAG_COD_1',
 'SOSP_DIAG_DESC_1',
 'TIPO_DIAG_2',
 'SOSP_DIAG_COD_2',
 'SOSP_DIAG_DESC_2',
 'TIPO_DIAG_3',
 'SOSP_DIAG_COD_3',
 'SOSP_DIAG_DESC_3',
 'TIPO_DIAG_4',
 'SOSP_DIAG_COD_4',
 'SOSP_DIAG_DESC_4',
 'TIPO_DIAG_5',
 'SOSP_DIAG_COD_5',
 'SOSP_DIAG_DESC_5',
 'ESTAB_DESTINO_COD',
 'ESTAB_DESTINO_DESC',
 'ESTAB_DESTINO_LINEA_ASIS',
 'ESPECIALIDAD_DESTINO',
 'PRIORIDAD_DESTINO',
 'CODIGO_P

In [186]:
#aux[aux['NRO_IDENTIFICADOR'] == '88259394'][['ESTADO_SOL', 'ESPECIALIDAD_DESTINO', 'FECHA_SOLIC']]

# Cruce BDD Gestoras HLF

In [187]:
resultado_filtrado_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
0,Cancer de Tiroides,3602046,6,LUIS RODRÍGUEZ SANDOVAL,09999108115,NaN,NaN,2023-05-25,CESFAM MAFIOLETTI,Endocrinología Adulto,Rechazada,None,NaN,"[Tumor maligno de la glándula tiroides, (de la...",[C73],NaN
1,Cancer de Tiroides,4537021,6,SILVIA ELIANA ALFARO ROBLES,56993374116,56993374116,NaN,2024-01-30,HLF,Cirugia Cabeza y Cuello,Rechazada,None,NaN,"[Tumor maligno de la glandula tiroides, Tumor ...","[J849, C34, C73]",NaN
2,Cancer de Tiroides,5133109,5,MARÍA LUISA MORÁN OLMOS,922825667,940669480,NaN,2023-12-15,HLF,Endocrinología Adulto,Rechazada,None,NaN,"[Insuficiencia respiratoria aguda, Tumor malig...","[C73, J960]",NaN
3,Cancer de Tiroides,5711967,5,MARÍA ELENA MONTENEGRO LIZANA,987418977,997780573,99660194,2024-03-04,HLF,Endocrinología Adulto,Rechazada,None,NaN,[Tumor maligno de la glandula tiroides],[C73],NaN
4,Cancer de Tiroides,5819360,7,JUAN ANTONIO SEGOVIA MUÑOZ,992774625,NaN,NaN,2024-05-03,HLF,Broncopulmonar Adulto,Solicitada,None,NaN,[Carcinoma in situ de la glandula tiroides y d...,[D093],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,Mieloma,9352492,6,HUGO EFRAÍN SUÁREZ LECAROS,42936121,42936121,NaN,2024-02-21,CESFAM SANTA AMALIA,Urología Adulto,Solicitada,None,NaN,"[Lesión de sitios contiguos de la vejiga, Lesi...",[C67.8],NaN
26,Mieloma,9571379,3,JUAN GONZALO RUBILAR CISTERNA,959877538,NaN,9988785701,2024-05-31,HLF,Cirugía Adulto,Solicitada,None,NaN,"[Tumor maligno de la vejiga urinaria, Hernia i...","[C67, K40]",NaN
27,Mieloma,10192892,6,PAULA IVONNE CAVIEDES CABRERA,0986261767,0991065511,NaN,2023-07-11,CESFAM SANTA AMALIA,Urología Adulto,Rechazada,None,NaN,"[Cáncer Vesical en Personas de 15 años y más, ...",[C67.9],NaN
28,Mieloma,15327501,7,LEONARDO PATRICIO VÁSQUEZ MONTECINO,0978984583,0999929431,NaN,2023-03-16,HLF,Urología Adulto,Rechazada,None,NaN,[Carcinoma Urotelial/vesical],[],NaN


In [188]:

dataframe = resultado_filtrado_2.copy()

# Obtener la ruta del directorio actual del script
#dir_actual = os.path.dirname(__file__)
#dir_actual = os.getcwd()

dir_actual = os.getcwd()


# Construir la ruta completa al archivo CSV
archivo_excel = os.path.join(dir_actual, '..','DERIVACIONES', 'BASE DE TRABAJO_DERIVACIONES.xlsx') # Gestion_Oncologica -> apolo

dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


# Obtener la cantidad de filas que no son nulas en la columna 'mi_columna'
cantidad_filas_no_nulas = dataframe_2['Rut'].count()



print("Cantidad de pacientes en la base", cantidad_filas_no_nulas)



dataframe_2 = dataframe_2[:cantidad_filas_no_nulas]

dataframe_2.reset_index(drop=True, inplace=True)


# Renombrar la columna 'old_name' a 'new_name'
dataframe_2 = dataframe_2.rename(columns={'Rut': 'RUT'})


dataframe_2 = dataframe_2.rename(columns={'Problema de Salud': 'Problema Oncologico'})

dataframe_2 = dataframe_2.rename(columns={'Nombre': 'Nombre del Paciente'})

# Definir una función para quitar los últimos dos dígitos
def quitar_ultimos_dos(elemento):
    return np.int64(elemento[:-2])

# Convertir todos los elementos de la columna 'RUT' a string
dataframe_2['RUT'] = dataframe_2['RUT'].astype(str)

# Aplicar la función a la columna 'RUT'
#dataframe_2['RUT'] = dataframe_2['RUT'].apply(quitar_ultimos_dos)



# Identificar las filas donde 'DV' es NaN
filas_dv_nan = dataframe_2[dataframe_2['DV'].isna()]

# Iterar sobre estas filas y corregir los valores de 'DV' y 'Rut'
for index, fila in filas_dv_nan.iterrows():
    rut = str(fila['RUT'])
    if True:
      dv = rut[-1]  # Último dígito del Rut
      dataframe_2.at[index, 'DV'] = dv
      dataframe_2.at[index, 'RUT'] = rut[:-1]  # Eliminar el último dígito del Rut

dataframe_2['RUT'] = dataframe_2['RUT'].astype(int)


Cantidad de pacientes en la base 0


# Cruce BDD Gestoras Local

In [189]:
#dataframe = resultado_filtrado_2.copy()

#dir_actual = os.getcwd()


# Construir la ruta relativa al directorio donde se encuentra el archivo CSV
#dir_archivo_xlsx = os.path.join(dir_actual, '..','..','Datos HLF','BDD Gestoras')

# Construir la ruta completa al archivo CSV
#archivo_excel = os.path.join(dir_archivo_xlsx, 'BASE DE TRABAJO_DERIVACIONES.xlsx') # Gestion_Oncologica -> apolo

#dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


#cantidad_filas_no_nulas = dataframe_2['Rut'].count()



#print("Cantidad de pacientes en la base", cantidad_filas_no_nulas)



#dataframe_2 = dataframe_2[:cantidad_filas_no_nulas]

#dataframe_2.reset_index(drop=True, inplace=True)


# Renombrar la columna 'old_name' a 'new_name'
#dataframe_2 = dataframe_2.rename(columns={'Rut': 'RUT'})


#dataframe_2 = dataframe_2.rename(columns={'Problema de Salud': 'Problema Oncologico'})

#dataframe_2 = dataframe_2.rename(columns={'Nombre': 'Nombre del Paciente'})

# Definir una función para quitar los últimos dos dígitos
#def quitar_ultimos_dos(elemento):
#    return np.int64(elemento[:-2])

# Convertir todos los elementos de la columna 'RUT' a string
#dataframe_2['RUT'] = dataframe_2['RUT'].astype(str)

# Aplicar la función a la columna 'RUT'
#dataframe_2['RUT'] = dataframe_2['RUT'].apply(quitar_ultimos_dos)



# Identificar las filas donde 'DV' es NaN
#filas_dv_nan = dataframe_2[dataframe_2['DV'].isna()]

# Iterar sobre estas filas y corregir los valores de 'DV' y 'Rut'
#for index, fila in filas_dv_nan.iterrows():
#    rut = str(fila['RUT'])
#    if True:
#      dv = rut[-1]  # Último dígito del Rut
#      dataframe_2.at[index, 'DV'] = dv
#      dataframe_2.at[index, 'RUT'] = rut[:-1]  # Eliminar el último dígito del Rut

#dataframe_2['RUT'] = dataframe_2['RUT'].astype(int)

In [190]:
dataframe_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico,Especialidad,Fecha de SIC,Estado de SIC,Establecimiento de Origen,Diagnosticos,Observaciones,Telefonos,CITAS FUTURAS,MEDICO,ESTADO SIC


In [191]:
dataframe

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
0,Cancer de Tiroides,3602046,6,LUIS RODRÍGUEZ SANDOVAL,09999108115,NaN,NaN,2023-05-25,CESFAM MAFIOLETTI,Endocrinología Adulto,Rechazada,None,NaN,"[Tumor maligno de la glándula tiroides, (de la...",[C73],NaN
1,Cancer de Tiroides,4537021,6,SILVIA ELIANA ALFARO ROBLES,56993374116,56993374116,NaN,2024-01-30,HLF,Cirugia Cabeza y Cuello,Rechazada,None,NaN,"[Tumor maligno de la glandula tiroides, Tumor ...","[J849, C34, C73]",NaN
2,Cancer de Tiroides,5133109,5,MARÍA LUISA MORÁN OLMOS,922825667,940669480,NaN,2023-12-15,HLF,Endocrinología Adulto,Rechazada,None,NaN,"[Insuficiencia respiratoria aguda, Tumor malig...","[C73, J960]",NaN
3,Cancer de Tiroides,5711967,5,MARÍA ELENA MONTENEGRO LIZANA,987418977,997780573,99660194,2024-03-04,HLF,Endocrinología Adulto,Rechazada,None,NaN,[Tumor maligno de la glandula tiroides],[C73],NaN
4,Cancer de Tiroides,5819360,7,JUAN ANTONIO SEGOVIA MUÑOZ,992774625,NaN,NaN,2024-05-03,HLF,Broncopulmonar Adulto,Solicitada,None,NaN,[Carcinoma in situ de la glandula tiroides y d...,[D093],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,Mieloma,9352492,6,HUGO EFRAÍN SUÁREZ LECAROS,42936121,42936121,NaN,2024-02-21,CESFAM SANTA AMALIA,Urología Adulto,Solicitada,None,NaN,"[Lesión de sitios contiguos de la vejiga, Lesi...",[C67.8],NaN
26,Mieloma,9571379,3,JUAN GONZALO RUBILAR CISTERNA,959877538,NaN,9988785701,2024-05-31,HLF,Cirugía Adulto,Solicitada,None,NaN,"[Tumor maligno de la vejiga urinaria, Hernia i...","[C67, K40]",NaN
27,Mieloma,10192892,6,PAULA IVONNE CAVIEDES CABRERA,0986261767,0991065511,NaN,2023-07-11,CESFAM SANTA AMALIA,Urología Adulto,Rechazada,None,NaN,"[Cáncer Vesical en Personas de 15 años y más, ...",[C67.9],NaN
28,Mieloma,15327501,7,LEONARDO PATRICIO VÁSQUEZ MONTECINO,0978984583,0999929431,NaN,2023-03-16,HLF,Urología Adulto,Rechazada,None,NaN,[Carcinoma Urotelial/vesical],[],NaN


# Merge Total pacientes vs Gestoras

In [192]:
# Ejemplo de creación de DataFrame (puedes cargar tus propios DataFrames)
df1 = dataframe.copy()
df2 = dataframe_2.copy()

# Agrega una columna para identificar el origen de cada RUT
df1['_df'] = 'df1'
df2['_df'] = 'df2'

# Convertir la columna 'RUT' en ambos DataFrames al mismo tipo de datos
df1['RUT'] = df1['RUT'].astype(str)
df2['RUT'] = df2['RUT'].astype(str)



# Realiza un merge de los dos DataFrames en la columna 'RUT' y conserva solo las filas que aparecen en 'left_only'
df_merge = pd.merge(df1, df2, on='RUT', how='outer', indicator=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Nombre del Paciente_x'].fillna(df_merge['Nombre del Paciente_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Nombre del Paciente_x': 'Nombre del Paciente'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['Nombre del Paciente_y'], inplace=True)

# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Problema Oncologico_x'].fillna(df_merge['Problema Oncologico_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Problema Oncologico_x': 'Problema Oncologico'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['Problema Oncologico_y'], inplace=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['DV_x'].fillna(df_merge['DV_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'DV_x': 'DV'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['DV_y'], inplace=True)

df_ca_tiroide = df_merge[df_merge['_merge']=='left_only']

# Reemplazar los valores NaN por '-'
df_ca_tiroide['N° Telefono 1'] = df_ca_tiroide['N° Telefono 1'].fillna('-')
df_ca_tiroide['N° Telefono 2'] = df_ca_tiroide['N° Telefono 2'].fillna('-')
df_ca_tiroide['N° Telefono 3'] = df_ca_tiroide['N° Telefono 3'].fillna('-')

def combinar_telefonos(row):
    return '/'.join(filter(None, [str(row['N° Telefono 1']), str(row['N° Telefono 2']), str(row['N° Telefono 3'])]))

# Aplicar la función a lo largo de las filas del DataFrame para crear la columna 'Telefono Consolidado'
df_ca_tiroide_telefono = df_ca_tiroide[['N° Telefono 1','N° Telefono 2','N° Telefono 3']]
df_ca_tiroide['Telefono Consolidado2'] = df_ca_tiroide_telefono.apply(combinar_telefonos, axis=1)

# Eliminar las columnas originales de los números de teléfono si ya no son necesarias
df_ca_tiroide.drop(columns=['N° Telefono 1', 'N° Telefono 2', 'N° Telefono 3'], inplace=True)

df_ca_tiroide['Etapa'] = np.nan

df_ca_tiroide['Estado'] = np.nan

df_ca_tiroide['Fecha Proximo Hito'] = np.nan

df_ca_tiroide['Observacion'] = np.nan

df_ca_tiroide['Medico Responsable'] = np.nan

df_ca_tiroide['Fecha Ultima atencion'] = np.nan

df_ca_tiroide['Profesional Ultima atencion'] = np.nan

df_ca_tiroide['MEDICO'] = np.nan

df_ca_tiroide['CITAS FUTURAS'] = np.nan


In [193]:
list(df_ca_tiroide.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Fecha de SIC_x',
 'Establecimiento de Origen_x',
 'Especialidad Destino',
 'Estado de SIC_x',
 'Nombre garantia',
 'Fecha Proximo Hito',
 'DIAGNOSTICOS',
 'CODIGOS_DIAGNOSTICO',
 'Nombre agenda Medico especialista',
 '_df_x',
 'Estado',
 'Fecha Próximo Hito',
 'Medico ',
 'Especialidad',
 'Fecha de SIC_y',
 'Estado de SIC_y',
 'Establecimiento de Origen_y',
 'Diagnosticos',
 'Observaciones',
 'Telefonos',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC',
 '_df_y',
 '_merge',
 'Telefono Consolidado2',
 'Etapa',
 'Observacion',
 'Medico Responsable',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']

In [194]:
list(df_ca_tiroide.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Fecha de SIC_x',
 'Establecimiento de Origen_x',
 'Especialidad Destino',
 'Estado de SIC_x',
 'Nombre garantia',
 'Fecha Proximo Hito',
 'DIAGNOSTICOS',
 'CODIGOS_DIAGNOSTICO',
 'Nombre agenda Medico especialista',
 '_df_x',
 'Estado',
 'Fecha Próximo Hito',
 'Medico ',
 'Especialidad',
 'Fecha de SIC_y',
 'Estado de SIC_y',
 'Establecimiento de Origen_y',
 'Diagnosticos',
 'Observaciones',
 'Telefonos',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC',
 '_df_y',
 '_merge',
 'Telefono Consolidado2',
 'Etapa',
 'Observacion',
 'Medico Responsable',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']

In [195]:

#df_ca_tiroide = df_ca_tiroide[['Problema Oncologico', 'RUT', 'DV', 'Nombre del Paciente','Etapa','Nombre garantia','Estado','Fecha Proximo Hito','Observacion','Medico Responsable','Fecha Ultima atencion', 'Profesional Ultima atencion','DIAGNOSTICOS', 'Telefono Consolidado2']]
df_ca_tiroide = df_ca_tiroide.rename(columns={'Fecha de SIC_x': 'Fecha de SIC'})
df_ca_tiroide = df_ca_tiroide.rename(columns={'Estado de SIC_x': 'Estado de SIC'})
df_ca_tiroide = df_ca_tiroide.rename(columns={'Establecimiento de Origen_x': 'Establecimiento de Origen'})

df_ca_tiroide = df_ca_tiroide[['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Especialidad Destino',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'DIAGNOSTICOS',
 'Observaciones',
 'Telefono Consolidado2',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC']]

In [196]:
df_ca_tiroide

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CITAS FUTURAS,MEDICO,ESTADO SIC
0,Cancer Renal,10030458,9,RAFAEL GUILLERMO RAYO SILVA,NaN,NaN,NaN,Dermatología Adulto,2023-01-18,Rechazada,HLF,"[ACIDOSIS TUBULAR RENAL DISTAL TIPO IV, Insufi...",NaN,930506868/0957819324/-,NaN,NaN,NaN
1,Cancer Colorectal,10102012,6,LORETO SIRKA REBOLLEDO ÁLVAREZ,NaN,NaN,NaN,Medicina Interna,2024-05-20,Citada,HLF,"[CÁNCER COLO-RECTAL, CANCER RECTO, CÁNCER DE R...",NaN,56994527129/56222861503/56990455602,NaN,NaN,NaN
2,Cancer Gástrico,10126888,8,JUAN JOSÉ BASUALTO CUEVAS,NaN,NaN,NaN,Cirugía Abdominal Adulto,2023-08-29,Rechazada,CESFAM TRINIDAD,"[Tumor maligno del fundus gástrico , Tumor mal...",NaN,09957396397/-/-,NaN,NaN,NaN
3,Mieloma,10133772,3,MARIO HERNÁN MUÑOZ PINO,NaN,NaN,NaN,Hematología Adulto,2024-05-10,Citada,HLF,"[Insuficiencia renal aguda, Sindrome nefrotico...",NaN,-/02228959852/-,NaN,NaN,NaN
4,Cancer de Tiroides,10155911,4,Bernabe Alfredo Delgado Martinez,NaN,NaN,NaN,Endocrinología Adulto,2024-05-02,Solicitada,CESFAM BELLAVISTA,[Cáncer de tiroides en personas de 15 años y m...,NaN,86091988/NO ESPECIFICADO/-,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,Cancer Colorectal,9878678,3,FRANCISCO JAVIER ESPINOZA JARA,NaN,NaN,NaN,Urología Adulto,2024-04-23,Citada,HLF,"[Colostomia, Tumor maligno del colon]",NaN,0986927343/979217464/-,NaN,NaN,NaN
714,Cancer Gástrico,9879781,5,AMADOR ANTONIO MARTÍNEZ OLIVARES,NaN,NaN,NaN,Cirugía Abdominal Adulto,2023-09-28,Citada,CESFAM TRINIDAD,[Tumor maligno de la curvatura menor del estóm...,NaN,78690420/0/-,NaN,NaN,NaN
715,Cancer Colorectal,9893459,6,CARMEN GLORIA VILCHES MANRÍQUEZ,NaN,NaN,NaN,Gastroenterología Adulto,2023-07-26,Solicitada,HLF,[Tumor benigno del colon ascendente],NaN,79823119/79823119/-,NaN,NaN,NaN
716,Cancer de Tiroides,9979513,1,MYRIAM LUZ JASKELMAN PÉREZ,NaN,NaN,NaN,Endocrinología Adulto,2023-01-06,Rechazada,CESFAM MAFIOLETTI,[Cáncer de tiroides en personas de 15 años y m...,NaN,96122217/96122217/-,NaN,NaN,NaN


# Guardar dataframe

In [198]:
# Guardar el DataFrame en un archivo de Excel

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_derivaciones_nuevas.xlsx'  # Nombre del archivo de Excel
df_ca_tiroide.to_excel(os.path.join(dir_actual, '..','DERIVACIONES','Reporte diario casos nuevos',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 

# Convertir la columna 'RUT' en DataFrame_2 a tipo object si es necesario


df_concatenated_2 = pd.concat([dataframe_2,df_ca_tiroide])

#print(list[df_concatenated_2.columns])




df_concatenated_2['RUT'] = df_concatenated_2['RUT'].astype(str)


In [114]:
list(df_concatenated_2.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico ',
 'Especialidad',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'Diagnosticos',
 'Observaciones',
 'Telefonos',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC',
 'Medico Responsable',
 'Especialidad Destino',
 'DIAGNOSTICOS',
 'Telefono Consolidado2']

In [66]:
df_concatenated_2['Establecimiento de Origen']

0      CESFAM LA FLORIDA
1                    HLF
2                    HLF
3        CESFAM TRINIDAD
4                    HLF
             ...        
713                  HLF
714      CESFAM TRINIDAD
715                  HLF
716    CESFAM MAFIOLETTI
717                  HLF
Name: Establecimiento de Origen, Length: 718, dtype: object

In [223]:
df_concatenated_2[df_concatenated_2['RUT'] == '10327175']

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico,Especialidad,Fecha de SIC,Estado de SIC,...,Diagnosticos,Observaciones,Telefonos,CITAS FUTURAS,MEDICO,ESTADO SIC,Medico Responsable,Especialidad Destino,DIAGNOSTICOS,Telefono Consolidado2
10,Cancer de Tiroides,10327175,4,JUANA ALICIA UMAÑA PINO,NaN,NaN,NaN,NaN,2023-11-29,Citada,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Urología Adulto,"[Tumor maligno de la glandula tiroides, Tumor ...",97209188/09945917931/-
11,Cancer Renal,10327175,4,JUANA ALICIA UMAÑA PINO,NaN,NaN,NaN,NaN,2023-11-29,Citada,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Urología Adulto,"[Tumor maligno de la glandula tiroides, Tumor ...",97209188/09945917931/-


# Crear Citas futuras

In [56]:

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)



ruta_del_archivo = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','AgendasFuturas', f'{today.year}{month_formatted}{day_formatted}_Citas_Futuras.csv')


# Carga el archivo XLSX en un DataFrame
dataframe_fu = cargar_dataframe_ambulatorio(ruta_del_archivo)

# Crear Citas futuras local

In [57]:
#dir_actual = os.getcwd()

# Construir la ruta relativa al directorio donde se encuentra el archivo CSV
#dir_archivo_csv = os.path.join(dir_actual, '..','..')

# Entrar a la carpeta "Datos HLF"
#dir_datos_hlf = os.path.join(dir_archivo_csv, 'Datos HLF')



#ruta_del_archivo = os.path.join(dir_datos_hlf, '20240502_Citas_Futuras.csv')


# Carga el archivo XLSX en un DataFrame
#dataframe_fu = cargar_dataframe_ambulatorio(ruta_del_archivo)

In [58]:
# Mover los nombres de las columnas una posición hacia la izquierda
new_columns = list(dataframe_fu.columns)
new_columns = new_columns[1:] + [new_columns[0]]
dataframe_fu.columns = new_columns


In [59]:

dataframe_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1716992099255310814,Procedimiento Dr. Slater,Otorrinolaringología,11-06-2024,800,Agendada,FibroNasFaringLaringoscopia AMB,CEX,N,N,NaN,...,88351846,88351846,NaN,NaN,JUAN DE DIOS MALEBRAN 3625 CASA 17,La Florida,88351846,88351846,NaN,NaN
1716992099255310814,Procedimiento Dr. Slater,Otorrinolaringología,11-06-2024,800,Agendada,FibroNasFaringLaringoscopia AMB,CEX,N,N,NaN,...,88351846,88351846,NaN,NaN,JUAN DE DIOS MALEBRAN 3625 CASA 17,La Florida,88351846,88351846,NaN,NaN
1717165730962310833,DR. ANNA ELIZABETH PALMAR SULBARAN,Traumatología Infantil,11-06-2024,910,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,...,84124150,NO ESPECIFICADO,NaN,NaN,teniente de marina rolando frodden,La Florida,84124150,NO ESPECIFICADO,NaN,NaN
1717165730962310833,DR. ANNA ELIZABETH PALMAR SULBARAN,Traumatología Infantil,11-06-2024,910,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,...,84124150,NO ESPECIFICADO,NaN,NaN,teniente de marina rolando frodden,La Florida,84124150,NO ESPECIFICADO,NaN,NaN
1717166885950310178,DR. ANA MARÍA ARAVENA PARAS,Traumatología Infantil,11-06-2024,1100,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,...,52239577,NO ESPECIFICADO,NaN,NaN,ALCAZAR,Puente Alto,52239577,NO ESPECIFICADO,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713451086016300967,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1000,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,922269900,922269900,NaN,NaN,LAS ACACIAS,La Florida,922269900,922269900,NaN,NaN
1713451086016300967,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1000,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,922269900,922269900,NaN,NaN,LAS ACACIAS,La Florida,922269900,922269900,NaN,NaN
1713453317365300554,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1040,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,967782271,951077542,NaN,NaN,VICUÑA MACKENNA,San Joaquín,967782271,951077542,NaN,NaN
1715610771664310021,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1140,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,0937564768,NaN,NaN,NaN,serrano,La Florida,0937564768,NaN,NaN,NaN


In [60]:
# Eliminar duplicados por índice y mantener el primero
df_sin_duplicados = dataframe_fu.drop_duplicates(keep='first')


In [61]:
df_sin_duplicados['ESTADO_CITA']

1716992099255310814    Agendada
1717165730962310833    Agendada
1717166885950310178    Agendada
1716813691556300544    Agendada
1716911107847300119    Agendada
                         ...   
1714662664397310870    Agendada
1713453317365300554    Agendada
1715609402723310398    Agendada
1713451086016300967    Agendada
1715610771664310021    Agendada
Name: ESTADO_CITA, Length: 26242, dtype: object

In [62]:
df_sin_duplicados['ESTADO_CITA'].unique()

array(['Agendada', 'Anulada', 'En buzón de reasignación'], dtype=object)

In [63]:
df_sin_duplicados[df_sin_duplicados['ESPECIALIDAD_AGENDA']=='Imagenología AMB']['DESCRIPCION']

1717804127490510198                             Ecografia Renal
1715698304345300078                           Rx Hombro Der F-L
1717702782856500862                           Rx Muñeca Izq F-L
1717702846897500871                           Rx Muñeca Der F-L
1716834561064300111                 TAC de Extremidades, sin MC
                                         ...                   
1716406735799300462                              Rx Pie Izq F-L
1712323640596910122                    Rx Cadera Der Crosstable
1712323464883910069    Rx Pelvis AP para evaluación de Prótesis
1716830805648300587                                Rx Pelvis AP
1703877427182900912               TC de Abdomen y Pelvis con MC
Name: DESCRIPCION, Length: 1336, dtype: object

In [64]:
list(df_sin_duplicados)

['NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO',
 'NOMBRE_TIPO_DOCUMENTO',
 'NUM_DOCUMENTO_TUTOR',
 'NOMBRE_DOCUMENTO_TUTOR',
 'CITAPACIFECHNACI',
 'SEXO',
 'NACIONALIDAD',
 'FACT_FINANCIADOR',
 'FACT_COBERTURA',
 'GES_EPISODIO',
 'MOVIL_HABITUAL',
 'TELEFONO1_HABITUAL',
 'TELEFONO2_HABITUAL',
 'TELEFONO3_HABITUALDIRECCION_HABITUAL',
 'COMUNA_HABITUAL',
 'MOVIL_CONTACTO',
 'TELEFONO1_CONTACTO',
 'TELEFONO2_CONTACTO',
 'TELEFONO3_CONTACTO',
 'CITA_KEY']

In [65]:
#regex_esp = re.compile(r'(?i)\bbroncopulmonar|cirug[íi]a.*(abdominal|adulto|cabeza.*cuello|t[óo]rax|digestiva|general)|coloproctolog[íi]a|dermatolog[íi]a|endocrinolog[íi]a|gastroenterolog[íi]a|hematolog[íi]a|medicina.*interna|traumatolog[íi]a|urolog[íi]a', re.IGNORECASE)

#dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]

In [66]:
df_sin_duplicados['ESPECIALIDAD_AGENDA'].unique()

array(['Otorrinolaringología', 'Traumatología Infantil',
       'Psiquiatría Infantil', 'Neurología Adulto',
       'Enfermería Multiespecialidad', 'Cirugía Proctológica',
       'Alto Riesgo Obstétrico', 'Broncopulmonar Adulto',
       'Hospital de Día', 'Infectología Infantil', 'Pediatría',
       'Imagenología AMB', 'Traumatología Adulto',
       'Cirugía Vascular Periférica', 'Terapia Ocupacional MFR',
       'Kinesiología MFR', 'Cirugía Adulto', 'Gastroenterología Adulto',
       'Oftalmología', 'Enfermería', 'Nutricionista Infantil',
       'Ginecología Adulto', 'Psiquiatría Adulto', 'Salud Ocupacional',
       'Unidad Alivio Dolor y Cuidados Paliativos', 'Cardiología Adulto',
       'Cirugía y Trauma. MáxiloFacial', 'Endodoncia', 'Cirugía Infantil',
       'Medicina Interna', 'Rehabilitación: Prótesis Removible',
       'Diabetes', 'Gastroenterología Infantil', 'Odontopediatría',
       'Nutricionista Adulto', 'Geriatría', 'Cardiología Infantil',
       'Endocrinología Adulto', 

In [67]:
regex_esp = re.compile(r'(?i)\bimagenolog[ií]a|\bbroncopulmonar|\bcirug[íi]a.*(abdominal|adulto|cabeza.*cuello|t[óo]rax|digestiva|general)|coloproctolog[íi]a|dermatolog[íi]a|endocrinolog[íi]a|gastroenterolog[íi]a|hematolog[íi]a|medicina.*interna|traumatolog[íi]a|urolog[íi]a', re.IGNORECASE)

endocrinologia_df_fu = df_sin_duplicados[df_sin_duplicados['ESPECIALIDAD_AGENDA'].str.contains(regex_esp, na=False)]

In [68]:
endocrinologia_df_fu['ESPECIALIDAD_AGENDA'].unique()

array(['Traumatología Infantil', 'Neurología Adulto',
       'Broncopulmonar Adulto', 'Imagenología AMB',
       'Traumatología Adulto', 'Cirugía Adulto',
       'Gastroenterología Adulto', 'Medicina Interna',
       'Gastroenterología Infantil', 'Endocrinología Adulto',
       'Urología Adulto', 'Broncopulmonar Infantil',
       'Endocrinología Infantil', 'Neurología Infantil',
       'Dermatología Adulto', 'Hematología Adulto',
       'Cirugía Abdominal Adulto', 'Cirugía Plástica Adulto',
       'Hematología Infantil', 'Cirugia Cabeza y Cuello',
       'Cirugía Tórax Adulto'], dtype=object)

In [69]:

    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
endocrinologia_df_fu = endocrinologia_df_fu[~endocrinologia_df_fu['ESTADO_CITA'].isin(['Anulada', 'En buzón de reasignación'])]

In [70]:
endocrinologia_df_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1717165730962310833,DR. ANNA ELIZABETH PALMAR SULBARAN,Traumatología Infantil,11-06-2024,910,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,...,84124150,NO ESPECIFICADO,NaN,NaN,teniente de marina rolando frodden,La Florida,84124150,NO ESPECIFICADO,NaN,NaN
1717166885950310178,DR. ANA MARÍA ARAVENA PARAS,Traumatología Infantil,11-06-2024,1100,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,...,52239577,NO ESPECIFICADO,NaN,NaN,ALCAZAR,Puente Alto,52239577,NO ESPECIFICADO,NaN,NaN
1716911107847300119,DR. GONZALO ANDRÉS FORES RAMÍREZ,Neurología Adulto,11-06-2024,1400,Agendada,Derivacion desde APS,PRI,S,S,Epilepsia no refractaria en personas de 15 año...,...,NO ESPECIFICADO,79016597,NaN,NaN,EL RETAMO,La Florida,NO ESPECIFICADO,79016597,NaN,NaN
1717426616193300484,DR. LAUTARO CLEMENTE CAMPOS TORRES,Traumatología Infantil,11-06-2024,800,Agendada,Ingreso Médico (transitorio),CEX,N,N,NaN,...,977778488,NaN,NaN,NaN,oje tres sur poniente,La Florida,977778488,NaN,NaN,NaN
1717682936080510665,JOSE RAMON SILVA FIGUEROA,Broncopulmonar Adulto,11-06-2024,1430,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,948507611,NaN,NaN,NaN,SANTA DELIA,La Florida,948507611,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711637800965910740,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1515,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,977998577,977998577,NaN,NaN,SINAI 9882,La Florida,977998577,977998577,NaN,NaN
1711570784927900514,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1545,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,56998623844,56988586700,NaN,NaN,Guacolda,La Florida,56998623844,56988586700,NaN,NaN
1711981070379900773,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1415,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,0987243463,02225166822,966980845,NaN,Pleamar,La Florida,0987243463,02225166822,966980845,NaN
1716830805648300587,Sala I-3,Imagenología AMB,12-11-2024,1100,Agendada,Rx Pelvis AP,CEX,N,N,NaN,...,090000000000,NaN,NaN,NaN,NaN,La Florida,090000000000,NaN,NaN,NaN


In [71]:
# Convertir la columna de fecha a formato datetime
#endocrinologia_df_fu['CITAFECH'] = pd.to_datetime(endocrinologia_df_fu['CITAFECH'])

# Formatear la columna de fecha en el formato deseado
#endocrinologia_df_fu['CITAFECH'] = endocrinologia_df_fu['CITAFECH'].dt.strftime('%Y-%m-%d')

In [72]:
endocrinologia_df_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1717165730962310833,DR. ANNA ELIZABETH PALMAR SULBARAN,Traumatología Infantil,11-06-2024,910,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,...,84124150,NO ESPECIFICADO,NaN,NaN,teniente de marina rolando frodden,La Florida,84124150,NO ESPECIFICADO,NaN,NaN
1717166885950310178,DR. ANA MARÍA ARAVENA PARAS,Traumatología Infantil,11-06-2024,1100,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,...,52239577,NO ESPECIFICADO,NaN,NaN,ALCAZAR,Puente Alto,52239577,NO ESPECIFICADO,NaN,NaN
1716911107847300119,DR. GONZALO ANDRÉS FORES RAMÍREZ,Neurología Adulto,11-06-2024,1400,Agendada,Derivacion desde APS,PRI,S,S,Epilepsia no refractaria en personas de 15 año...,...,NO ESPECIFICADO,79016597,NaN,NaN,EL RETAMO,La Florida,NO ESPECIFICADO,79016597,NaN,NaN
1717426616193300484,DR. LAUTARO CLEMENTE CAMPOS TORRES,Traumatología Infantil,11-06-2024,800,Agendada,Ingreso Médico (transitorio),CEX,N,N,NaN,...,977778488,NaN,NaN,NaN,oje tres sur poniente,La Florida,977778488,NaN,NaN,NaN
1717682936080510665,JOSE RAMON SILVA FIGUEROA,Broncopulmonar Adulto,11-06-2024,1430,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,948507611,NaN,NaN,NaN,SANTA DELIA,La Florida,948507611,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711637800965910740,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1515,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,977998577,977998577,NaN,NaN,SINAI 9882,La Florida,977998577,977998577,NaN,NaN
1711570784927900514,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1545,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,56998623844,56988586700,NaN,NaN,Guacolda,La Florida,56998623844,56988586700,NaN,NaN
1711981070379900773,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1415,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,0987243463,02225166822,966980845,NaN,Pleamar,La Florida,0987243463,02225166822,966980845,NaN
1716830805648300587,Sala I-3,Imagenología AMB,12-11-2024,1100,Agendada,Rx Pelvis AP,CEX,N,N,NaN,...,090000000000,NaN,NaN,NaN,NaN,La Florida,090000000000,NaN,NaN,NaN


In [73]:
endocrinologia_df_fu['DV'] = endocrinologia_df_fu['DOCUMENTO'].str[-1]

endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['DOCUMENTO'].str[:-1]

In [74]:
endocrinologia_df_fu = endocrinologia_df_fu[endocrinologia_df_fu['NOMBRE_TIPO_DOCUMENTO']=='RUN']

In [75]:
endocrinologia_df_fu['NOMBRE_TIPO_DOCUMENTO']

1717165730962310833    RUN
1717166885950310178    RUN
1716911107847300119    RUN
1717426616193300484    RUN
1717682936080510665    RUN
                      ... 
1711637800965910740    RUN
1711570784927900514    RUN
1711981070379900773    RUN
1716830805648300587    RUN
1703877427182900912    RUN
Name: NOMBRE_TIPO_DOCUMENTO, Length: 8086, dtype: object

In [76]:
endocrinologia_df_fu['RUT'].value_counts()

RUT
22087761    11
25775319    10
7481630      9
6244508      9
12133848     8
            ..
23055792     1
20987435     1
22504257     1
22493298     1
6830435      1
Name: count, Length: 6504, dtype: int64

In [77]:
list(endocrinologia_df_fu.columns)

['NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO',
 'NOMBRE_TIPO_DOCUMENTO',
 'NUM_DOCUMENTO_TUTOR',
 'NOMBRE_DOCUMENTO_TUTOR',
 'CITAPACIFECHNACI',
 'SEXO',
 'NACIONALIDAD',
 'FACT_FINANCIADOR',
 'FACT_COBERTURA',
 'GES_EPISODIO',
 'MOVIL_HABITUAL',
 'TELEFONO1_HABITUAL',
 'TELEFONO2_HABITUAL',
 'TELEFONO3_HABITUALDIRECCION_HABITUAL',
 'COMUNA_HABITUAL',
 'MOVIL_CONTACTO',
 'TELEFONO1_CONTACTO',
 'TELEFONO2_CONTACTO',
 'TELEFONO3_CONTACTO',
 'CITA_KEY',
 'DV',
 'RUT']

In [78]:
endocrinologia_df_fu[endocrinologia_df_fu['RUT']=='7073448']

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY,DV,RUT
1716827669781310027,KLGO. TANNIA DENISSE MATAMALA CHACANA,Broncopulmonar Adulto,11-06-2024,1130,Agendada,Rehabilitación Pulmonar-P.Func.AMB,CEX,N,N,NaN,...,NaN,NaN,LOS CAZADORES,La Florida,85008857,23588841,NaN,NaN,6,7073448
1716827856077310081,KLGO. ANAÍS CASTRO ALFARO,Broncopulmonar Adulto,14-06-2024,1100,Agendada,Rehabilitación Pulmonar-P.Func.AMB,CEX,N,N,NaN,...,NaN,NaN,LOS CAZADORES,La Florida,85008857,23588841,NaN,NaN,6,7073448
1716827673667310031,KLGO. TANNIA DENISSE MATAMALA CHACANA,Broncopulmonar Adulto,18-06-2024,1130,Agendada,Rehabilitación Pulmonar-P.Func.AMB,CEX,N,N,NaN,...,NaN,NaN,LOS CAZADORES,La Florida,85008857,23588841,NaN,NaN,6,7073448
1716827860456310086,KLGO. ANAÍS CASTRO ALFARO,Broncopulmonar Adulto,21-06-2024,1100,Agendada,Rehabilitación Pulmonar-P.Func.AMB,CEX,N,N,NaN,...,NaN,NaN,LOS CAZADORES,La Florida,85008857,23588841,NaN,NaN,6,7073448
1716827677317310034,KLGO. TANNIA DENISSE MATAMALA CHACANA,Broncopulmonar Adulto,25-06-2024,1130,Agendada,Rehabilitación Pulmonar-P.Func.AMB,CEX,N,N,NaN,...,NaN,NaN,LOS CAZADORES,La Florida,85008857,23588841,NaN,NaN,6,7073448
1716827865444310089,KLGO. ANAÍS CASTRO ALFARO,Broncopulmonar Adulto,28-06-2024,1100,Agendada,Rehabilitación Pulmonar-P.Func.AMB,CEX,N,N,NaN,...,NaN,NaN,LOS CAZADORES,La Florida,85008857,23588841,NaN,NaN,6,7073448


In [79]:
endocrinologia_df_fu = endocrinologia_df_fu[endocrinologia_df_fu['NOMBRE_AGENDA'].str.contains(r'\bDR.')]

In [80]:
endocrinologia_df_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY,DV,RUT
1717165730962310833,DR. ANNA ELIZABETH PALMAR SULBARAN,Traumatología Infantil,11-06-2024,910,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,...,NaN,NaN,teniente de marina rolando frodden,La Florida,84124150,NO ESPECIFICADO,NaN,NaN,K,28311984
1717166885950310178,DR. ANA MARÍA ARAVENA PARAS,Traumatología Infantil,11-06-2024,1100,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,...,NaN,NaN,ALCAZAR,Puente Alto,52239577,NO ESPECIFICADO,NaN,NaN,4,28406730
1716911107847300119,DR. GONZALO ANDRÉS FORES RAMÍREZ,Neurología Adulto,11-06-2024,1400,Agendada,Derivacion desde APS,PRI,S,S,Epilepsia no refractaria en personas de 15 año...,...,NaN,NaN,EL RETAMO,La Florida,NO ESPECIFICADO,79016597,NaN,NaN,K,10062768
1717426616193300484,DR. LAUTARO CLEMENTE CAMPOS TORRES,Traumatología Infantil,11-06-2024,800,Agendada,Ingreso Médico (transitorio),CEX,N,N,NaN,...,NaN,NaN,oje tres sur poniente,La Florida,977778488,NaN,NaN,NaN,9,22461563
1717165547614310799,DR. ANNA ELIZABETH PALMAR SULBARAN,Traumatología Infantil,11-06-2024,850,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,...,NaN,NaN,STA EDITH,La Florida,27301138,0,NaN,NaN,3,41584426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711570697187900499,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,17-10-2024,1515,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,NaN,NaN,Psje Huentelauquen,La Florida,997842276,26149648,NaN,NaN,K,5316627
1711548687149910674,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1400,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,NaN,NaN,rio pudeto,Puente Alto,74879538,74879538,NaN,NaN,0,6849972
1711637800965910740,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1515,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,NaN,NaN,SINAI 9882,La Florida,977998577,977998577,NaN,NaN,3,5812444
1711570784927900514,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1545,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,NaN,NaN,Guacolda,La Florida,56998623844,56988586700,NaN,NaN,0,3727939


In [81]:
endocrinologia_df_fu['RUT'].value_counts()

RUT
10298920    3
5122647     3
28339540    3
6115427     3
26013370    3
           ..
6624325     1
6875646     1
7157185     1
7284083     1
7893201     1
Name: count, Length: 5230, dtype: int64

In [82]:
endocrinologia_df_fu[endocrinologia_df_fu['RUT']=='3460934']

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY,DV,RUT
1713879964147300042,DR. JOAN CRISTÓBAL BASSA MOYANO,Urología Adulto,05-07-2024,1015,Agendada,Control postquirurgico,CEX,N,N,NaN,...,NaN,NaN,Monseñor Subercaseaux,La Florida,98298342,994550664,NaN,NaN,9,3460934
1712924212278300379,DR. MAXIMILIANO EUGENIO NARVÁEZ FUENTES,Urología Adulto,06-09-2024,1400,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,NaN,NaN,Monseñor Subercaseaux,La Florida,98298342,994550664,NaN,NaN,9,3460934


In [83]:
endocrinologia_df_fu[endocrinologia_df_fu['RUT']=='16269122']

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY,DV,RUT


In [84]:
endocrinologia_df_fu[endocrinologia_df_fu['RUT']=='21154687']

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY,DV,RUT
1712673871543100017,DR. EVA ISABEL RETAMAL RIQUELME,Neurología Adulto,25-06-2024,1000,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,NaN,NaN,CUATRO NORTE,La Granja,982319578,NaN,NaN,NaN,5,21154687


In [85]:
endocrinologia_df_fu[endocrinologia_df_fu['RUT']=='6277674']

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY,DV,RUT
1710166726506700921,DR. PATRICIA MARISOL ARIAS CAMPOVERDE,Broncopulmonar Adulto,12-06-2024,1400,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,NaN,NaN,LIRQUEN,La Granja,52813728,25117075,NaN,NaN,9,6277674
1711462208334900788,DR. MAXIMILIANO EUGENIO NARVÁEZ FUENTES,Urología Adulto,28-06-2024,1345,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,NaN,NaN,LIRQUEN,La Granja,52813728,25117075,NaN,NaN,9,6277674


In [86]:
list(endocrinologia_df_fu.columns)

['NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO',
 'NOMBRE_TIPO_DOCUMENTO',
 'NUM_DOCUMENTO_TUTOR',
 'NOMBRE_DOCUMENTO_TUTOR',
 'CITAPACIFECHNACI',
 'SEXO',
 'NACIONALIDAD',
 'FACT_FINANCIADOR',
 'FACT_COBERTURA',
 'GES_EPISODIO',
 'MOVIL_HABITUAL',
 'TELEFONO1_HABITUAL',
 'TELEFONO2_HABITUAL',
 'TELEFONO3_HABITUALDIRECCION_HABITUAL',
 'COMUNA_HABITUAL',
 'MOVIL_CONTACTO',
 'TELEFONO1_CONTACTO',
 'TELEFONO2_CONTACTO',
 'TELEFONO3_CONTACTO',
 'CITA_KEY',
 'DV',
 'RUT']

In [87]:
endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['RUT'].astype(int)

In [88]:
endocrinologia_df_fu = endocrinologia_df_fu[['RUT','NOMBRE_AGENDA','ESPECIALIDAD_AGENDA','CITAFECH','CITAHORA','ESTADO_CITA','DESCRIPCION','ORIGEN','GES_CITA','GES_SOL','PATOLOGIA_GES_SOL','PACIENTE','CITA_CODUTEN','DOCUMENTO']]

In [89]:
endocrinologia_df_fu

,RUT,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,PACIENTE,CITA_CODUTEN,DOCUMENTO
1717165730962310833,28311984,DR. ANNA ELIZABETH PALMAR SULBARAN,Traumatología Infantil,11-06-2024,910,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,Jacinta Belen Toledo Flores,AC00546528,28311984K
1717166885950310178,28406730,DR. ANA MARÍA ARAVENA PARAS,Traumatología Infantil,11-06-2024,1100,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,Celeste Andromeda Pascale Borquez Rodriguez,AC00546530,284067304
1716911107847300119,10062768,DR. GONZALO ANDRÉS FORES RAMÍREZ,Neurología Adulto,11-06-2024,1400,Agendada,Derivacion desde APS,PRI,S,S,Epilepsia no refractaria en personas de 15 año...,Alexis Vera Gonzalez,AC00546648,10062768K
1717426616193300484,22461563,DR. LAUTARO CLEMENTE CAMPOS TORRES,Traumatología Infantil,11-06-2024,800,Agendada,Ingreso Médico (transitorio),CEX,N,N,NaN,THOMAS ANDRÉS VALLEJOS BELTRÁN,AC00547359,224615639
1717165547614310799,41584426,DR. ANNA ELIZABETH PALMAR SULBARAN,Traumatología Infantil,11-06-2024,850,Agendada,Derivacion desde APS,PRI,S,S,Displasia luxante de caderas,Adhara Carranza Montilla,AC00544292,415844263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711570697187900499,5316627,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,17-10-2024,1515,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,ANTONIO ENRIQUE BAEZA ALEGRÍA,AC00026674,5316627K
1711548687149910674,6849972,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1400,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,LUIS SEREY VEGA,AC00041717,68499720
1711637800965910740,5812444,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1515,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,VÍCTOR ENRIQUE GONZÁLEZ ESCOBAR,AC00485558,58124443
1711570784927900514,3727939,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1545,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,JOSÉ MARCELO CASTILLO LAVOZ,AC00234789,37279390


In [90]:
# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)


In [95]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(endocrinologia_df_fu, index= False, header = True):
    ws.append(r)

for cell in ws['D'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_citas_futuras_derivaciones.xlsx'
wb.save(os.path.join(dir_actual, '..','DERIVACIONES',f'{nombre_archivo}'))

In [91]:
#nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_citas_futuras_derivaciones.xlsx'  # Nombre del archivo de Excel
#endocrinologia_df_fu.to_excel(nombre_archivo, index=False) 


In [92]:
#endocrinologia_df_fu['DV'] = endocrinologia_df_fu['DOCUMENTO'].str[-1]

#endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['DOCUMENTO'].str[:-1]

In [93]:
dataframe_final_2 = df_concatenated_2.merge(endocrinologia_df_fu, on='RUT', how='left')